In [ ]:
import sys
import os

# Add the path to your StyleGAN2 repository
stylegan2_path = os.path.abspath('stylegan2')
sys.path.append('/Users/folder name or path/stylegan2')

import dnnlib
#import legacy

In [ ]:
import argparse
import cv2
import numpy as np
from PIL import Image, ImageEnhance
from scipy.ndimage import gaussian_filter, map_coordinates
from skimage import exposure
import os
import random

# Define image augmentation functions

def skew_image(image, skew_factor):
    rows, cols, _ = image.shape
    M = np.float32([[1, skew_factor, 0], [0, 1, 0]])
    skewed = cv2.warpAffine(image, M, (cols, rows))
    return skewed

def rotate_image(image, angle):
    rows, cols, _ = image.shape
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    rotated = cv2.warpAffine(image, M, (cols, rows))
    return rotated

def translate_image(image, tx, ty):
    rows, cols, _ = image.shape
    M = np.float32([[1, 0, tx * cols], [0, 1, ty * rows]])
    translated = cv2.warpAffine(image, M, (cols, rows))
    return translated

def scale_image(image, scale):
    rows, cols, _ = image.shape
    M = np.float32([[scale, 0, 0], [0, scale, 0]])
    new_cols, new_rows = int(cols * scale), int(rows * scale)
    scaled = cv2.warpAffine(image, M, (new_cols, new_rows), borderMode=cv2.BORDER_REFLECT)
    return scaled

def shear_image(image, shear):
    rows, cols, _ = image.shape
    M = np.float32([[1, shear, 0], [0, 1, 0]])
    sheared = cv2.warpAffine(image, M, (cols, rows))
    return sheared

def flip_image(image, direction):
    if direction == 'horizontal':
        flipped = cv2.flip(image, 1)
    elif direction == 'vertical':
        flipped = cv2.flip(image, 0)
    else:
        raise ValueError("Direction must be 'horizontal' or 'vertical'")
    return flipped

def zoom_image(image, zoom_factor):
    rows, cols, _ = image.shape
    center = (cols // 2, rows // 2)
    M = cv2.getRotationMatrix2D(center, 0, zoom_factor)
    zoomed = cv2.warpAffine(image, M, (cols, rows))
    return zoomed

def add_gaussian_noise(image, var):
    """
    Add Gaussian noise to an image.

    :param image: Input image as a NumPy array with values in [0, 1].
    :param var: Variance of the Gaussian noise.
    :return: Noisy image.
    """
    mean = 0
    sigma = var**0.5
    gaussian = np.random.normal(mean, sigma, image.shape)
    noisy = np.clip(image + gaussian, 0, 1)
    return noisy

def add_salt_and_pepper_noise(image, amount=0.05):
    """
    Add salt-and-pepper noise to an image.

    :param image: Input image as a NumPy array with values in [0, 1].
    :param amount: Proportion of pixels to be replaced with salt and pepper noise.
    :return: Noisy image.
    """
    if len(image.shape) != 3:
        raise ValueError("Image must be 3-dimensional.")
    
    rows, cols, _ = image.shape
    s_vs_p = 0.5
    out = np.copy(image)
    
    # Salt noise
    num_salt = np.ceil(amount * image.size * s_vs_p)
    salt_coords = [np.random.randint(0, i, int(num_salt)) for i in image.shape[:2]]
    out[salt_coords[0], salt_coords[1]] = 1
    
    # Pepper noise
    num_pepper = np.ceil(amount * image.size * (1. - s_vs_p))
    pepper_coords = [np.random.randint(0, i, int(num_pepper)) for i in image.shape[:2]]
    out[pepper_coords[0], pepper_coords[1]] = 0
    
    return out

def apply_gaussian_blur(image, sigma):
    blurred = gaussian_filter(image, sigma=sigma)
    return blurred

def sharpen_image(image):
    if image.dtype == np.float32 or image.dtype == np.float64:
        # Normalize image if it's not in 0-255 range
        image = (image * 255).astype(np.uint8)
    
    # Define a sharpening kernel
    kernel = np.array([[0, -0.25, 0], 
                       [-0.25, 2, -0.25], 
                       [0, -0.25, 0]], dtype=np.float32)
    
    sharpened = cv2.filter2D(image, -1, kernel)
    
    if sharpened.dtype == np.uint8:
        # Normalize back to [0, 1] if necessary
        sharpened = sharpened / 255.0
    
    return sharpened

def temperature_jitter(image, jitter_amount):
    image = Image.fromarray((image * 255).astype(np.uint8))
    enhancer = ImageEnhance.Color(image)
    jittered = enhancer.enhance(1 + jitter_amount)
    jittered = np.array(jittered) / 255.0
    return jittered

def grid_mask(image, grid_size=30):
    rows, cols, _ = image.shape
    mask = np.ones((rows, cols), dtype=np.uint8)
    
    for r in range(0, rows, grid_size):
        for c in range(0, cols, grid_size):
            mask[r:min(r + grid_size, rows), c:min(c + grid_size, cols)] = 0

    grid_masked_image = np.copy(image)
    grid_masked_image[mask == 0] = 0
    return grid_masked_image

def elastic_distortion(image, alpha=1.0, sigma=0.1):
    """
    Apply elastic distortion to an image.

    :param image: Input image as a NumPy array.
    :param alpha: Scaling factor for the displacement field.
    :param sigma: Standard deviation for the Gaussian filter.
    :return: Distorted image.
    """
    shape = image.shape
    height, width = shape[:2]
    
    # Create displacement fields
    dx = gaussian_filter((np.random.rand(height, width) * 2 - 1), sigma, mode='constant', cval=0.0) * alpha
    dy = gaussian_filter((np.random.rand(height, width) * 2 - 1), sigma, mode='constant', cval=0.0) * alpha

    # Generate meshgrid
    x, y = np.meshgrid(np.arange(width), np.arange(height))
    x = x + dx
    y = y + dy
    
    # Ensure coordinates are within bounds
    x = np.clip(x, 0, width - 1)
    y = np.clip(y, 0, height - 1)
    
    # Apply elastic distortion to each channel
    distorted_image = np.zeros_like(image)
    for i in range(image.shape[2]):  # Assuming image has 3 channels
        distorted_image[..., i] = map_coordinates(image[..., i], [y.flatten(), x.flatten()], order=1, mode='reflect').reshape(image.shape[:2])

    return distorted_image

def pseudocoloring(image):
    img = (image * 255).astype(np.uint8)
    pseudocolored = cv2.applyColorMap(img, cv2.COLORMAP_JET)
    return pseudocolored / 255.0

def random_cropping(image, crop_size):
    h, w, _ = image.shape
    crop_h, crop_w = crop_size
    if crop_h > h or crop_w > w:
        raise ValueError("Crop size must be smaller than the dimensions of the image.")
    x = np.random.randint(0, w - crop_w + 1)
    y = np.random.randint(0, h - crop_h + 1)
    cropped = image[y:y + crop_h, x:x + crop_w]
    return cropped

# Placeholder functions
def mosaic(image, size=32):
    rows, cols, _ = image.shape
    mosaic_image = np.copy(image)
    
    for r in range(0, rows, size):
        for c in range(0, cols, size):
            if r + size <= rows and c + size <= cols:
                block = image[r:r + size, c:c + size]
                mean_color = np.mean(block, axis=(0, 1))
                mosaic_image[r:r + size, c:c + size] = mean_color
    
    return mosaic_image

def blend_images(original, mosaic, alpha=0.5):
    return cv2.addWeighted(original, alpha, mosaic, 1 - alpha, 0)

def bounding_box(image, box_color=(0, 255, 0), box_thickness=3):
    rows, cols, _ = image.shape
    box_image = np.copy(image)
    cv2.rectangle(box_image, (10, 10), (cols-10, rows-10), box_color, box_thickness)
    return box_image

def collar_jitter(image, collar_size=0.1):
    rows, cols, _ = image.shape
    collar_width = int(min(rows, cols) * collar_size)
    collar_image = np.copy(image)
    collar_image[:collar_width, :] = np.random.rand(collar_width, cols, 3)
    collar_image[-collar_width:, :] = np.random.rand(collar_width, cols, 3)
    collar_image[:, :collar_width] = np.random.rand(rows, collar_width, 3)
    collar_image[:, -collar_width:] = np.random.rand(rows, collar_width, 3)
    return collar_image

def image_warp(image, alpha=1.0):
    rows, cols, _ = image.shape
    M = np.float32([[1, alpha, 0], [0, 1, 0]])
    warped_image = cv2.warpAffine(image, M, (cols, rows), flags=cv2.INTER_LINEAR)
    return warped_image

def channel_shuffle(image):
    shuffled_image = np.copy(image)
    channels = [0, 1, 2]
    np.random.shuffle(channels)
    return shuffled_image[..., channels]

def polar_distortion(image):
    rows, cols, _ = image.shape
    center = (cols // 2, rows // 2)
    max_radius = np.sqrt((center[0] ** 2) + (center[1] ** 2))
    distorted = np.zeros_like(image)
    for r in range(rows):
        for c in range(cols):
            radius = np.sqrt((c - center[0]) ** 2 + (r - center[1]) ** 2)
            theta = np.arctan2(r - center[1], c - center[0])
            radius = radius / max_radius
            x = int(center[0] + radius * np.cos(theta) * center[0])
            y = int(center[1] + radius * np.sin(theta) * center[1])
            if 0 <= x < cols and 0 <= y < rows:
                distorted[r, c] = image[y, x]
    return distorted

def hide_and_seek(image):
    rows, cols, _ = image.shape
    hide_image = np.copy(image)
    mask = np.random.choice([0, 1], size=(rows, cols), p=[0.2, 0.8])
    hide_image[mask == 0] = 0
    return hide_image

def apply_box_grid(image, grid_size=32):
    rows, cols, _ = image.shape
    grid_image = np.copy(image)

    for r in range(0, rows, grid_size):
        for c in range(0, cols, grid_size):
            if r + grid_size <= rows and c + grid_size <= cols:
                cv2.rectangle(grid_image, (c, r), (c + grid_size, r + grid_size), (0, 1, 0), 1)
    
    return grid_image

def color_shift(image, shift_range=0.2):
    shift = np.random.uniform(-shift_range, shift_range, size=(3,))
    shifted_image = np.clip(image + shift, 0, 1)
    return shifted_image

def affine_transform(image, matrix=None):
    rows, cols, _ = image.shape
    if matrix is None:
        matrix = np.float32([[1, 0, 0], [0, 1, 0]])
    transformed_image = cv2.warpAffine(image, matrix, (cols, rows), flags=cv2.INTER_LINEAR)
    return transformed_image

def deformable_conv(image):
    kernel = np.array([[0, 0, 0], [0, 1, 0], [0, 0, 0]], dtype=np.float32)
    deformable_image = cv2.filter2D(image, -1, kernel)
    return deformable_image

def solarize(image, threshold):
    image = (image * 255).astype(np.uint8)
    solarized = cv2.bitwise_not(image) if np.mean(image) > threshold * 255 else image
    return solarized / 255.0

def invert_image(image):
    return 1.0 - image

def color_jitter(image):
    img = Image.fromarray((image * 255).astype(np.uint8))
    enhancer = ImageEnhance.Color(img)
    jittered = enhancer.enhance(np.random.uniform(0.5, 1.5))
    jittered = np.array(jittered) / 255.0
    return jittered

def sigmoid_contrast(image):
    img = (image * 255).astype(np.uint8)
    img = exposure.adjust_sigmoid(img, gain=10, cutoff=0.5)
    return img / 255.0

def gamma_contrast(image):
    gamma = np.random.uniform(0.5, 2.0)
    img = (image * 255).astype(np.uint8)
    img = exposure.adjust_gamma(img, gamma)
    return img / 255.0

def linear_contrast(image):
    img = (image * 255).astype(np.uint8)
    img = exposure.rescale_intensity(img, in_range='image', out_range='dtype')
    return img / 255.0

def perspective_transform(image, src_pts=None, dst_pts=None):
    """Apply a perspective transformation to an image."""
    h, w = image.shape[:2]
    if src_pts is None or dst_pts is None:
        src_pts = np.float32([[0, 0], [w-1, 0], [0, h-1], [w-1, h-1]])
        dst_pts = np.float32([[0, 0], [w-1, 0], [0, h-1], [w-1, h-1]])
    
    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
    transformed_image = cv2.warpPerspective(image, matrix, (w, h))
    return transformed_image

def spatial_transform(image, theta):
    rows, cols, _ = image.shape
    M = np.float32([[1, 0, 0], [0, 1, 0]])
    transformed = cv2.warpAffine(image, M, (cols, rows), flags=cv2.INTER_LINEAR)
    return transformed

def mixup(image, image2):
    if image.shape != image2.shape:
        raise ValueError("Images must have the same dimensions for mixup.")
    
    alpha = np.random.uniform(0.3, 0.7)
    
    # Make sure both images are in the range [0, 1]
    image = np.clip(image, 0, 1)
    image2 = np.clip(image2, 0, 1)
    
    mixed = alpha * image + (1 - alpha) * image2
    
    # Ensure mixed image is still in the range [0, 1]
    mixed = np.clip(mixed, 0, 1)
    
    return mixed

def random_erasing(image, scale=(0.02, 0.4), ratio=(0.3, 3.3), value=0.0):
    """
    Applies random erasing to the given image.

    Args:
        image (np.ndarray): The input image to be processed.
        scale (tuple): Range of the erasing area as a fraction of the image area.
        ratio (tuple): Aspect ratio range of the erasing area.
        value (float or int): The value to fill the erased area (for grayscale images).

    Returns:
        np.ndarray: The processed image with random erasing applied.
    """
    rows, cols, _ = image.shape
    area = rows * cols

    # Determine the target area for erasing
    target_area = np.random.uniform(scale[0], scale[1]) * area
    aspect_ratio = np.random.uniform(ratio[0], ratio[1])

    erase_w = int(np.sqrt(target_area * aspect_ratio))
    erase_h = int(np.sqrt(target_area / aspect_ratio))

    # Ensure erasing dimensions are within image size
    erase_w = min(erase_w, cols)
    erase_h = min(erase_h, rows)

    if erase_w <= 0 or erase_h <= 0:
        print("Invalid erased area dimensions. Skipping random erasing.")
        return image

    print(f"Erase dimensions: width={erase_w}, height={erase_h}")

    # Randomly select the top-left corner of the erasing area
    x1 = np.random.randint(0, cols - erase_w + 1)
    y1 = np.random.randint(0, rows - erase_h + 1)

    # Apply the erasing
    image[y1:y1 + erase_h, x1:x1 + erase_w] = value

    return image

def occlusion(image, box_size):
    h, w, _ = image.shape
    if box_size > min(h, w):
        raise ValueError("Box size must be smaller than both dimensions of the image.")
    
    x = np.random.randint(0, w - box_size)
    y = np.random.randint(0, h - box_size)
    
    # Ensure the box is within image boundaries
    x = np.clip(x, 0, w - box_size)
    y = np.clip(y, 0, h - box_size)
    
    image[y:y + box_size, x:x + box_size, :] = 0
    return image

def apply_augmentations_to_image(image, image_name, output_dir):
    augmentations = [
        ('Skew', lambda img: skew_image(img, skew_factor=np.random.uniform(-0.2, 0.2))),
        ('Rotate', lambda img: rotate_image(img, angle=np.random.uniform(-30, 30))),
        ('Translate', lambda img: translate_image(img, tx=np.random.uniform(-0.2, 0.2), ty=np.random.uniform(-0.2, 0.2))),
        ('Scale', lambda img: scale_image(img, scale=np.random.uniform(0.8, 1.2))),
        ('Shear', lambda img: shear_image(img, shear=np.random.uniform(-0.2, 0.2))),
        ('Flip', lambda img: flip_image(img, direction=np.random.choice(['horizontal', 'vertical']))),
        ('Zoom', lambda img: zoom_image(img, zoom_factor=np.random.uniform(0.8, 1.2))),
        ('GaussianNoise', lambda img: add_gaussian_noise(img, var=np.random.uniform(0.01, 0.1))),
        ('SaltAndPepper', lambda img: add_salt_and_pepper_noise(img, amount=np.random.uniform(0.01, 0.1))),
        ('Blur', lambda img: apply_gaussian_blur(img, sigma=np.random.uniform(0.5, 2.0))),
        ('Sharpen', lambda img: sharpen_image(img)),
        ('TemperatureJitter', lambda img: temperature_jitter(img, jitter_amount=np.random.uniform(-0.2, 0.2))),
        ('Occlusion', lambda img: occlusion(np.copy(img), box_size=np.random.randint(10, 50))),
        ('ElasticDistortion', lambda img: elastic_distortion(img, alpha=np.random.uniform(1, 5), sigma=np.random.uniform(0.5, 2.0))),
        ('Pseudocoloring', lambda img: pseudocoloring(img)),
        ('RandomCropping', lambda img: random_cropping(img, crop_size=(np.random.randint(50, 100), np.random.randint(50, 100)))),
        ('Mosaic', lambda img: blend_images(img, mosaic(img, size=32), alpha=0.5)),
        ('BoundingBox', lambda img: bounding_box(img)),
        ('CollarJitter', lambda img: collar_jitter(img, collar_size=np.random.uniform(0.05, 0.2))),
        ('ImageWarp', lambda img: image_warp(img, alpha=np.random.uniform(0.1, 0.5))),
        ('ChannelShuffle', lambda img: channel_shuffle(img)),
        ('Solarize', lambda img: solarize(img, threshold=np.random.uniform(0.2, 0.8))),
        ('Invert', lambda img: invert_image(img)),
        ('ColorJitter', lambda img: color_jitter(img)),
        ('SigmoidContrast', lambda img: sigmoid_contrast(img)),
        ('GammaContrast', lambda img: gamma_contrast(img)),
        ('LinearContrast', lambda img: linear_contrast(img)),
        ('PolarDistortion', lambda img: polar_distortion(img)),
        ('HideAndSeek', lambda img: hide_and_seek(img)),
        ('BoxGrid', lambda img: apply_box_grid(img, grid_size=32)),
        ('Mixup', lambda img: mixup(img, image2=np.random.rand(*img.shape))),
        ('ColorShift', lambda img: color_shift(img)),
        ('AffineTransform', lambda img: affine_transform(img)),
        ('PerspectiveTransform', lambda img: perspective_transform(img, 
             src_pts=np.float32([[0, 0], [img.shape[1]-1, 0], [0, img.shape[0]-1], [img.shape[1]-1, img.shape[0]-1]]),
             dst_pts=np.float32([[0, 0], [img.shape[1]-1, 100], [0, img.shape[0]-100], [img.shape[1]-1, img.shape[0]-100]]))),
        ('SpatialTransform', lambda img: spatial_transform(img, theta=np.random.uniform(-0.1, 0.1))),
        ('DeformableConv', lambda img: deformable_conv(img)),
        ('RandomErasing', lambda img: random_erasing(img, scale=(0.02, 0.4), ratio=(0.3, 3.3), value=0.0))
    ]
    
    for i, (aug_name, aug_fn) in enumerate(augmentations):
        try:
            augmented_image = aug_fn(np.copy(image))  # Apply augmentations on a copy of the image
            output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}_{aug_name}_{i}.jpg")
            augmented_image = (augmented_image * 255).astype(np.uint8)
            augmented_image = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(output_path, augmented_image)
            print(f"Augmented image saved to {output_path}")
        except Exception as e:
            print(f"Error applying {aug_name}: {e}")

def process_image(input_image_path, output_dir):
    try:
        image = cv2.imread(input_image_path)
        if image is None:
            raise FileNotFoundError(f"Image not found: {input_image_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0  # Convert BGR to RGB and normalize
        image_name = os.path.basename(input_image_path)
        
        # Create a random second image with the same dimensions
        image2 = np.random.rand(*image.shape)  # Create random image with same shape
        image2 = np.clip(image2, 0, 1)  # Ensure image2 is in the range [0, 1]
        
        # Apply augmentations including mixup
        apply_augmentations_to_image(image, image_name, output_dir)
        
        # Apply mixup separately
        mixed_image = mixup(image, image2)
        output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}_Mixup.jpg")
        mixed_image = (mixed_image * 255).astype(np.uint8)
        mixed_image = cv2.cvtColor(mixed_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_path, mixed_image)
        print(f"Mixup image saved to {output_path}")
    
    except Exception as e:
        print(f"Error processing image {input_image_path}: {e}")
        
def test_augmentation(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename in os.listdir(input_dir):
        input_image_path = os.path.join(input_dir, filename)
        if os.path.isfile(input_image_path):
            print(f"Processing {input_image_path}...")
            process_image(input_image_path, output_dir)

# Set the paths to your directories
input_dir = ' # Update with the actual dataset path'
output_dir = '# update with the path or folder where it has to be saved'

# Process all images in the input directory and save them to the output directory
test_augmentation(input_dir, output_dir)

In [ ]:
import argparse
import cv2
import numpy as np
from PIL import Image, ImageEnhance
from scipy.ndimage import gaussian_filter, map_coordinates
from skimage import exposure
import os
import random

# Define image augmentation functions

def skew_image(image, skew_factor):
    rows, cols, _ = image.shape
    M = np.float32([[1, skew_factor, 0], [0, 1, 0]])
    skewed = cv2.warpAffine(image, M, (cols, rows))
    return skewed

def rotate_image(image, angle):
    rows, cols, _ = image.shape
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    rotated = cv2.warpAffine(image, M, (cols, rows))
    return rotated

def translate_image(image, tx, ty):
    rows, cols, _ = image.shape
    M = np.float32([[1, 0, tx * cols], [0, 1, ty * rows]])
    translated = cv2.warpAffine(image, M, (cols, rows))
    return translated

def scale_image(image, scale):
    rows, cols, _ = image.shape
    M = np.float32([[scale, 0, 0], [0, scale, 0]])
    new_cols, new_rows = int(cols * scale), int(rows * scale)
    scaled = cv2.warpAffine(image, M, (new_cols, new_rows), borderMode=cv2.BORDER_REFLECT)
    return scaled

def shear_image(image, shear):
    rows, cols, _ = image.shape
    M = np.float32([[1, shear, 0], [0, 1, 0]])
    sheared = cv2.warpAffine(image, M, (cols, rows))
    return sheared

def flip_image(image, direction):
    if direction == 'horizontal':
        flipped = cv2.flip(image, 1)
    elif direction == 'vertical':
        flipped = cv2.flip(image, 0)
    else:
        raise ValueError("Direction must be 'horizontal' or 'vertical'")
    return flipped

def zoom_image(image, zoom_factor):
    rows, cols, _ = image.shape
    center = (cols // 2, rows // 2)
    M = cv2.getRotationMatrix2D(center, 0, zoom_factor)
    zoomed = cv2.warpAffine(image, M, (cols, rows))
    return zoomed

def add_gaussian_noise(image, var):
    """
    Add Gaussian noise to an image.

    :param image: Input image as a NumPy array with values in [0, 1].
    :param var: Variance of the Gaussian noise.
    :return: Noisy image.
    """
    mean = 0
    sigma = var**0.5
    gaussian = np.random.normal(mean, sigma, image.shape)
    noisy = np.clip(image + gaussian, 0, 1)
    return noisy

def add_salt_and_pepper_noise(image, amount=0.05):
    """
    Add salt-and-pepper noise to an image.

    :param image: Input image as a NumPy array with values in [0, 1].
    :param amount: Proportion of pixels to be replaced with salt and pepper noise.
    :return: Noisy image.
    """
    if len(image.shape) != 3:
        raise ValueError("Image must be 3-dimensional.")
    
    rows, cols, _ = image.shape
    s_vs_p = 0.5
    out = np.copy(image)
    
    # Salt noise
    num_salt = np.ceil(amount * image.size * s_vs_p)
    salt_coords = [np.random.randint(0, i, int(num_salt)) for i in image.shape[:2]]
    out[salt_coords[0], salt_coords[1]] = 1
    
    # Pepper noise
    num_pepper = np.ceil(amount * image.size * (1. - s_vs_p))
    pepper_coords = [np.random.randint(0, i, int(num_pepper)) for i in image.shape[:2]]
    out[pepper_coords[0], pepper_coords[1]] = 0
    
    return out

def apply_gaussian_blur(image, sigma):
    blurred = gaussian_filter(image, sigma=sigma)
    return blurred

def sharpen_image(image):
    if image.dtype == np.float32 or image.dtype == np.float64:
        # Normalize image if it's not in 0-255 range
        image = (image * 255).astype(np.uint8)
    
    # Define a sharpening kernel
    kernel = np.array([[0, -0.25, 0], 
                       [-0.25, 2, -0.25], 
                       [0, -0.25, 0]], dtype=np.float32)
    
    sharpened = cv2.filter2D(image, -1, kernel)
    
    if sharpened.dtype == np.uint8:
        # Normalize back to [0, 1] if necessary
        sharpened = sharpened / 255.0
    
    return sharpened

def temperature_jitter(image, jitter_amount):
    image = Image.fromarray((image * 255).astype(np.uint8))
    enhancer = ImageEnhance.Color(image)
    jittered = enhancer.enhance(1 + jitter_amount)
    jittered = np.array(jittered) / 255.0
    return jittered

def grid_mask(image, grid_size=30):
    rows, cols, _ = image.shape
    mask = np.ones((rows, cols), dtype=np.uint8)
    
    for r in range(0, rows, grid_size):
        for c in range(0, cols, grid_size):
            mask[r:min(r + grid_size, rows), c:min(c + grid_size, cols)] = 0

    grid_masked_image = np.copy(image)
    grid_masked_image[mask == 0] = 0
    return grid_masked_image

def elastic_distortion(image, alpha=1.0, sigma=0.1):
    """
    Apply elastic distortion to an image.

    :param image: Input image as a NumPy array.
    :param alpha: Scaling factor for the displacement field.
    :param sigma: Standard deviation for the Gaussian filter.
    :return: Distorted image.
    """
    shape = image.shape
    height, width = shape[:2]
    
    # Create displacement fields
    dx = gaussian_filter((np.random.rand(height, width) * 2 - 1), sigma, mode='constant', cval=0.0) * alpha
    dy = gaussian_filter((np.random.rand(height, width) * 2 - 1), sigma, mode='constant', cval=0.0) * alpha

    # Generate meshgrid
    x, y = np.meshgrid(np.arange(width), np.arange(height))
    x = x + dx
    y = y + dy
    
    # Ensure coordinates are within bounds
    x = np.clip(x, 0, width - 1)
    y = np.clip(y, 0, height - 1)
    
    # Apply elastic distortion to each channel
    distorted_image = np.zeros_like(image)
    for i in range(image.shape[2]):  # Assuming image has 3 channels
        distorted_image[..., i] = map_coordinates(image[..., i], [y.flatten(), x.flatten()], order=1, mode='reflect').reshape(image.shape[:2])

    return distorted_image

def pseudocoloring(image):
    img = (image * 255).astype(np.uint8)
    pseudocolored = cv2.applyColorMap(img, cv2.COLORMAP_JET)
    return pseudocolored / 255.0

def random_cropping(image, crop_size):
    h, w, _ = image.shape
    crop_h, crop_w = crop_size
    if crop_h > h or crop_w > w:
        raise ValueError("Crop size must be smaller than the dimensions of the image.")
    x = np.random.randint(0, w - crop_w + 1)
    y = np.random.randint(0, h - crop_h + 1)
    cropped = image[y:y + crop_h, x:x + crop_w]
    return cropped

# Placeholder functions
def mosaic(image, size=32):
    rows, cols, _ = image.shape
    mosaic_image = np.copy(image)
    
    for r in range(0, rows, size):
        for c in range(0, cols, size):
            if r + size <= rows and c + size <= cols:
                block = image[r:r + size, c:c + size]
                mean_color = np.mean(block, axis=(0, 1))
                mosaic_image[r:r + size, c:c + size] = mean_color
    
    return mosaic_image

def blend_images(original, mosaic, alpha=0.5):
    return cv2.addWeighted(original, alpha, mosaic, 1 - alpha, 0)

def bounding_box(image, box_color=(0, 255, 0), box_thickness=3):
    rows, cols, _ = image.shape
    box_image = np.copy(image)
    cv2.rectangle(box_image, (10, 10), (cols-10, rows-10), box_color, box_thickness)
    return box_image

def collar_jitter(image, collar_size=0.1):
    rows, cols, _ = image.shape
    collar_width = int(min(rows, cols) * collar_size)
    collar_image = np.copy(image)
    collar_image[:collar_width, :] = np.random.rand(collar_width, cols, 3)
    collar_image[-collar_width:, :] = np.random.rand(collar_width, cols, 3)
    collar_image[:, :collar_width] = np.random.rand(rows, collar_width, 3)
    collar_image[:, -collar_width:] = np.random.rand(rows, collar_width, 3)
    return collar_image

def image_warp(image, alpha=1.0):
    rows, cols, _ = image.shape
    M = np.float32([[1, alpha, 0], [0, 1, 0]])
    warped_image = cv2.warpAffine(image, M, (cols, rows), flags=cv2.INTER_LINEAR)
    return warped_image

def channel_shuffle(image):
    shuffled_image = np.copy(image)
    channels = [0, 1, 2]
    np.random.shuffle(channels)
    return shuffled_image[..., channels]

def polar_distortion(image):
    rows, cols, _ = image.shape
    center = (cols // 2, rows // 2)
    max_radius = np.sqrt((center[0] ** 2) + (center[1] ** 2))
    distorted = np.zeros_like(image)
    for r in range(rows):
        for c in range(cols):
            radius = np.sqrt((c - center[0]) ** 2 + (r - center[1]) ** 2)
            theta = np.arctan2(r - center[1], c - center[0])
            radius = radius / max_radius
            x = int(center[0] + radius * np.cos(theta) * center[0])
            y = int(center[1] + radius * np.sin(theta) * center[1])
            if 0 <= x < cols and 0 <= y < rows:
                distorted[r, c] = image[y, x]
    return distorted

def hide_and_seek(image):
    rows, cols, _ = image.shape
    hide_image = np.copy(image)
    mask = np.random.choice([0, 1], size=(rows, cols), p=[0.2, 0.8])
    hide_image[mask == 0] = 0
    return hide_image

def apply_box_grid(image, grid_size=32):
    rows, cols, _ = image.shape
    grid_image = np.copy(image)

    for r in range(0, rows, grid_size):
        for c in range(0, cols, grid_size):
            if r + grid_size <= rows and c + grid_size <= cols:
                cv2.rectangle(grid_image, (c, r), (c + grid_size, r + grid_size), (0, 1, 0), 1)
    
    return grid_image

def color_shift(image, shift_range=0.2):
    shift = np.random.uniform(-shift_range, shift_range, size=(3,))
    shifted_image = np.clip(image + shift, 0, 1)
    return shifted_image

def affine_transform(image, matrix=None):
    rows, cols, _ = image.shape
    if matrix is None:
        matrix = np.float32([[1, 0, 0], [0, 1, 0]])
    transformed_image = cv2.warpAffine(image, matrix, (cols, rows), flags=cv2.INTER_LINEAR)
    return transformed_image

def deformable_conv(image):
    kernel = np.array([[0, 0, 0], [0, 1, 0], [0, 0, 0]], dtype=np.float32)
    deformable_image = cv2.filter2D(image, -1, kernel)
    return deformable_image

def solarize(image, threshold):
    image = (image * 255).astype(np.uint8)
    solarized = cv2.bitwise_not(image) if np.mean(image) > threshold * 255 else image
    return solarized / 255.0

def invert_image(image):
    return 1.0 - image

def color_jitter(image):
    img = Image.fromarray((image * 255).astype(np.uint8))
    enhancer = ImageEnhance.Color(img)
    jittered = enhancer.enhance(np.random.uniform(0.5, 1.5))
    jittered = np.array(jittered) / 255.0
    return jittered

def sigmoid_contrast(image):
    img = (image * 255).astype(np.uint8)
    img = exposure.adjust_sigmoid(img, gain=10, cutoff=0.5)
    return img / 255.0

def gamma_contrast(image):
    gamma = np.random.uniform(0.5, 2.0)
    img = (image * 255).astype(np.uint8)
    img = exposure.adjust_gamma(img, gamma)
    return img / 255.0

def linear_contrast(image):
    img = (image * 255).astype(np.uint8)
    img = exposure.rescale_intensity(img, in_range='image', out_range='dtype')
    return img / 255.0

def perspective_transform(image, src_pts=None, dst_pts=None):
    """Apply a perspective transformation to an image."""
    h, w = image.shape[:2]
    if src_pts is None or dst_pts is None:
        src_pts = np.float32([[0, 0], [w-1, 0], [0, h-1], [w-1, h-1]])
        dst_pts = np.float32([[0, 0], [w-1, 0], [0, h-1], [w-1, h-1]])
    
    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
    transformed_image = cv2.warpPerspective(image, matrix, (w, h))
    return transformed_image

def spatial_transform(image, theta):
    rows, cols, _ = image.shape
    M = np.float32([[1, 0, 0], [0, 1, 0]])
    transformed = cv2.warpAffine(image, M, (cols, rows), flags=cv2.INTER_LINEAR)
    return transformed

def mixup(image, image2):
    if image.shape != image2.shape:
        raise ValueError("Images must have the same dimensions for mixup.")
    
    alpha = np.random.uniform(0.3, 0.7)
    
    # Make sure both images are in the range [0, 1]
    image = np.clip(image, 0, 1)
    image2 = np.clip(image2, 0, 1)
    
    mixed = alpha * image + (1 - alpha) * image2
    
    # Ensure mixed image is still in the range [0, 1]
    mixed = np.clip(mixed, 0, 1)
    
    return mixed

def random_erasing(image, scale=(0.02, 0.4), ratio=(0.3, 3.3), value=0.0):
    """
    Applies random erasing to the given image.

    Args:
        image (np.ndarray): The input image to be processed.
        scale (tuple): Range of the erasing area as a fraction of the image area.
        ratio (tuple): Aspect ratio range of the erasing area.
        value (float or int): The value to fill the erased area (for grayscale images).

    Returns:
        np.ndarray: The processed image with random erasing applied.
    """
    rows, cols, _ = image.shape
    area = rows * cols

    # Determine the target area for erasing
    target_area = np.random.uniform(scale[0], scale[1]) * area
    aspect_ratio = np.random.uniform(ratio[0], ratio[1])

    erase_w = int(np.sqrt(target_area * aspect_ratio))
    erase_h = int(np.sqrt(target_area / aspect_ratio))

    # Ensure erasing dimensions are within image size
    erase_w = min(erase_w, cols)
    erase_h = min(erase_h, rows)

    if erase_w <= 0 or erase_h <= 0:
        print("Invalid erased area dimensions. Skipping random erasing.")
        return image

    print(f"Erase dimensions: width={erase_w}, height={erase_h}")

    # Randomly select the top-left corner of the erasing area
    x1 = np.random.randint(0, cols - erase_w + 1)
    y1 = np.random.randint(0, rows - erase_h + 1)

    # Apply the erasing
    image[y1:y1 + erase_h, x1:x1 + erase_w] = value

    return image

def occlusion(image, box_size):
    h, w, _ = image.shape
    if box_size > min(h, w):
        raise ValueError("Box size must be smaller than both dimensions of the image.")
    
    x = np.random.randint(0, w - box_size)
    y = np.random.randint(0, h - box_size)
    
    # Ensure the box is within image boundaries
    x = np.clip(x, 0, w - box_size)
    y = np.clip(y, 0, h - box_size)
    
    image[y:y + box_size, x:x + box_size, :] = 0
    return image

def apply_augmentations_to_image(image, image_name, output_dir):
    augmentations = [
        ('Skew', lambda img: skew_image(img, skew_factor=np.random.uniform(-0.2, 0.2))),
        ('Rotate', lambda img: rotate_image(img, angle=np.random.uniform(-30, 30))),
        ('Translate', lambda img: translate_image(img, tx=np.random.uniform(-0.2, 0.2), ty=np.random.uniform(-0.2, 0.2))),
        ('Scale', lambda img: scale_image(img, scale=np.random.uniform(0.8, 1.2))),
        ('Shear', lambda img: shear_image(img, shear=np.random.uniform(-0.2, 0.2))),
        ('Flip', lambda img: flip_image(img, direction=np.random.choice(['horizontal', 'vertical']))),
        ('Zoom', lambda img: zoom_image(img, zoom_factor=np.random.uniform(0.8, 1.2))),
        ('GaussianNoise', lambda img: add_gaussian_noise(img, var=np.random.uniform(0.01, 0.1))),
        ('SaltAndPepper', lambda img: add_salt_and_pepper_noise(img, amount=np.random.uniform(0.01, 0.1))),
        ('Blur', lambda img: apply_gaussian_blur(img, sigma=np.random.uniform(0.5, 2.0))),
        ('Sharpen', lambda img: sharpen_image(img)),
        ('TemperatureJitter', lambda img: temperature_jitter(img, jitter_amount=np.random.uniform(-0.2, 0.2))),
        ('Occlusion', lambda img: occlusion(np.copy(img), box_size=np.random.randint(10, 50))),
        ('ElasticDistortion', lambda img: elastic_distortion(img, alpha=np.random.uniform(1, 5), sigma=np.random.uniform(0.5, 2.0))),
        ('Pseudocoloring', lambda img: pseudocoloring(img)),
        ('RandomCropping', lambda img: random_cropping(img, crop_size=(np.random.randint(50, 100), np.random.randint(50, 100)))),
        ('Mosaic', lambda img: blend_images(img, mosaic(img, size=32), alpha=0.5)),
        ('BoundingBox', lambda img: bounding_box(img)),
        ('CollarJitter', lambda img: collar_jitter(img, collar_size=np.random.uniform(0.05, 0.2))),
        ('ImageWarp', lambda img: image_warp(img, alpha=np.random.uniform(0.1, 0.5))),
        ('ChannelShuffle', lambda img: channel_shuffle(img)),
        ('Solarize', lambda img: solarize(img, threshold=np.random.uniform(0.2, 0.8))),
        ('Invert', lambda img: invert_image(img)),
        ('ColorJitter', lambda img: color_jitter(img)),
        ('SigmoidContrast', lambda img: sigmoid_contrast(img)),
        ('GammaContrast', lambda img: gamma_contrast(img)),
        ('LinearContrast', lambda img: linear_contrast(img)),
        ('PolarDistortion', lambda img: polar_distortion(img)),
        ('HideAndSeek', lambda img: hide_and_seek(img)),
        ('BoxGrid', lambda img: apply_box_grid(img, grid_size=32)),
        ('Mixup', lambda img: mixup(img, image2=np.random.rand(*img.shape))),
        ('ColorShift', lambda img: color_shift(img)),
        ('AffineTransform', lambda img: affine_transform(img)),
        ('PerspectiveTransform', lambda img: perspective_transform(img, 
             src_pts=np.float32([[0, 0], [img.shape[1]-1, 0], [0, img.shape[0]-1], [img.shape[1]-1, img.shape[0]-1]]),
             dst_pts=np.float32([[0, 0], [img.shape[1]-1, 100], [0, img.shape[0]-100], [img.shape[1]-1, img.shape[0]-100]]))),
        ('SpatialTransform', lambda img: spatial_transform(img, theta=np.random.uniform(-0.1, 0.1))),
        ('DeformableConv', lambda img: deformable_conv(img)),
        ('RandomErasing', lambda img: random_erasing(img, scale=(0.02, 0.4), ratio=(0.3, 3.3), value=0.0))
    ]
    
    for i, (aug_name, aug_fn) in enumerate(augmentations):
        try:
            augmented_image = aug_fn(np.copy(image))  # Apply augmentations on a copy of the image
            output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}_{aug_name}_{i}.jpg")
            augmented_image = (augmented_image * 255).astype(np.uint8)
            augmented_image = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(output_path, augmented_image)
            print(f"Augmented image saved to {output_path}")
        except Exception as e:
            print(f"Error applying {aug_name}: {e}")

def process_image(input_image_path, output_dir):
    try:
        image = cv2.imread(input_image_path)
        if image is None:
            raise FileNotFoundError(f"Image not found: {input_image_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0  # Convert BGR to RGB and normalize
        image_name = os.path.basename(input_image_path)
        
        # Create a random second image with the same dimensions
        image2 = np.random.rand(*image.shape)  # Create random image with same shape
        image2 = np.clip(image2, 0, 1)  # Ensure image2 is in the range [0, 1]
        
        # Apply augmentations including mixup
        apply_augmentations_to_image(image, image_name, output_dir)
        
        # Apply mixup separately
        mixed_image = mixup(image, image2)
        output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}_Mixup.jpg")
        mixed_image = (mixed_image * 255).astype(np.uint8)
        mixed_image = cv2.cvtColor(mixed_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_path, mixed_image)
        print(f"Mixup image saved to {output_path}")
    
    except Exception as e:
        print(f"Error processing image {input_image_path}: {e}")
        
def test_augmentation(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename in os.listdir(input_dir):
        input_image_path = os.path.join(input_dir, filename)
        if os.path.isfile(input_image_path):
            print(f"Processing {input_image_path}...")
            process_image(input_image_path, output_dir)

# Set the paths to your directories
input_dir = ' # Update with the actual dataset path'
output_dir = '# update with the path or folder where it has to be saved'

# Process all images in the input directory and save them to the output directory
test_augmentation(input_dir, output_dir)

In [ ]:
import argparse
import cv2
import numpy as np
from PIL import Image, ImageEnhance
from scipy.ndimage import gaussian_filter, map_coordinates
from skimage import exposure
import os
import random

# Define image augmentation functions

def skew_image(image, skew_factor):
    rows, cols, _ = image.shape
    M = np.float32([[1, skew_factor, 0], [0, 1, 0]])
    skewed = cv2.warpAffine(image, M, (cols, rows))
    return skewed

def rotate_image(image, angle):
    rows, cols, _ = image.shape
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    rotated = cv2.warpAffine(image, M, (cols, rows))
    return rotated

def translate_image(image, tx, ty):
    rows, cols, _ = image.shape
    M = np.float32([[1, 0, tx * cols], [0, 1, ty * rows]])
    translated = cv2.warpAffine(image, M, (cols, rows))
    return translated

def scale_image(image, scale):
    rows, cols, _ = image.shape
    M = np.float32([[scale, 0, 0], [0, scale, 0]])
    new_cols, new_rows = int(cols * scale), int(rows * scale)
    scaled = cv2.warpAffine(image, M, (new_cols, new_rows), borderMode=cv2.BORDER_REFLECT)
    return scaled

def shear_image(image, shear):
    rows, cols, _ = image.shape
    M = np.float32([[1, shear, 0], [0, 1, 0]])
    sheared = cv2.warpAffine(image, M, (cols, rows))
    return sheared

def flip_image(image, direction):
    if direction == 'horizontal':
        flipped = cv2.flip(image, 1)
    elif direction == 'vertical':
        flipped = cv2.flip(image, 0)
    else:
        raise ValueError("Direction must be 'horizontal' or 'vertical'")
    return flipped

def zoom_image(image, zoom_factor):
    rows, cols, _ = image.shape
    center = (cols // 2, rows // 2)
    M = cv2.getRotationMatrix2D(center, 0, zoom_factor)
    zoomed = cv2.warpAffine(image, M, (cols, rows))
    return zoomed

def add_gaussian_noise(image, var):
    """
    Add Gaussian noise to an image.

    :param image: Input image as a NumPy array with values in [0, 1].
    :param var: Variance of the Gaussian noise.
    :return: Noisy image.
    """
    mean = 0
    sigma = var**0.5
    gaussian = np.random.normal(mean, sigma, image.shape)
    noisy = np.clip(image + gaussian, 0, 1)
    return noisy

def add_salt_and_pepper_noise(image, amount=0.05):
    """
    Add salt-and-pepper noise to an image.

    :param image: Input image as a NumPy array with values in [0, 1].
    :param amount: Proportion of pixels to be replaced with salt and pepper noise.
    :return: Noisy image.
    """
    if len(image.shape) != 3:
        raise ValueError("Image must be 3-dimensional.")
    
    rows, cols, _ = image.shape
    s_vs_p = 0.5
    out = np.copy(image)
    
    # Salt noise
    num_salt = np.ceil(amount * image.size * s_vs_p)
    salt_coords = [np.random.randint(0, i, int(num_salt)) for i in image.shape[:2]]
    out[salt_coords[0], salt_coords[1]] = 1
    
    # Pepper noise
    num_pepper = np.ceil(amount * image.size * (1. - s_vs_p))
    pepper_coords = [np.random.randint(0, i, int(num_pepper)) for i in image.shape[:2]]
    out[pepper_coords[0], pepper_coords[1]] = 0
    
    return out

def apply_gaussian_blur(image, sigma):
    blurred = gaussian_filter(image, sigma=sigma)
    return blurred

def sharpen_image(image):
    if image.dtype == np.float32 or image.dtype == np.float64:
        # Normalize image if it's not in 0-255 range
        image = (image * 255).astype(np.uint8)
    
    # Define a sharpening kernel
    kernel = np.array([[0, -0.25, 0], 
                       [-0.25, 2, -0.25], 
                       [0, -0.25, 0]], dtype=np.float32)
    
    sharpened = cv2.filter2D(image, -1, kernel)
    
    if sharpened.dtype == np.uint8:
        # Normalize back to [0, 1] if necessary
        sharpened = sharpened / 255.0
    
    return sharpened

def temperature_jitter(image, jitter_amount):
    image = Image.fromarray((image * 255).astype(np.uint8))
    enhancer = ImageEnhance.Color(image)
    jittered = enhancer.enhance(1 + jitter_amount)
    jittered = np.array(jittered) / 255.0
    return jittered

def grid_mask(image, grid_size=30):
    rows, cols, _ = image.shape
    mask = np.ones((rows, cols), dtype=np.uint8)
    
    for r in range(0, rows, grid_size):
        for c in range(0, cols, grid_size):
            mask[r:min(r + grid_size, rows), c:min(c + grid_size, cols)] = 0

    grid_masked_image = np.copy(image)
    grid_masked_image[mask == 0] = 0
    return grid_masked_image

def elastic_distortion(image, alpha=1.0, sigma=0.1):
    """
    Apply elastic distortion to an image.

    :param image: Input image as a NumPy array.
    :param alpha: Scaling factor for the displacement field.
    :param sigma: Standard deviation for the Gaussian filter.
    :return: Distorted image.
    """
    shape = image.shape
    height, width = shape[:2]
    
    # Create displacement fields
    dx = gaussian_filter((np.random.rand(height, width) * 2 - 1), sigma, mode='constant', cval=0.0) * alpha
    dy = gaussian_filter((np.random.rand(height, width) * 2 - 1), sigma, mode='constant', cval=0.0) * alpha

    # Generate meshgrid
    x, y = np.meshgrid(np.arange(width), np.arange(height))
    x = x + dx
    y = y + dy
    
    # Ensure coordinates are within bounds
    x = np.clip(x, 0, width - 1)
    y = np.clip(y, 0, height - 1)
    
    # Apply elastic distortion to each channel
    distorted_image = np.zeros_like(image)
    for i in range(image.shape[2]):  # Assuming image has 3 channels
        distorted_image[..., i] = map_coordinates(image[..., i], [y.flatten(), x.flatten()], order=1, mode='reflect').reshape(image.shape[:2])

    return distorted_image

def pseudocoloring(image):
    img = (image * 255).astype(np.uint8)
    pseudocolored = cv2.applyColorMap(img, cv2.COLORMAP_JET)
    return pseudocolored / 255.0

def random_cropping(image, crop_size):
    h, w, _ = image.shape
    crop_h, crop_w = crop_size
    if crop_h > h or crop_w > w:
        raise ValueError("Crop size must be smaller than the dimensions of the image.")
    x = np.random.randint(0, w - crop_w + 1)
    y = np.random.randint(0, h - crop_h + 1)
    cropped = image[y:y + crop_h, x:x + crop_w]
    return cropped

# Placeholder functions
def mosaic(image, size=32):
    rows, cols, _ = image.shape
    mosaic_image = np.copy(image)
    
    for r in range(0, rows, size):
        for c in range(0, cols, size):
            if r + size <= rows and c + size <= cols:
                block = image[r:r + size, c:c + size]
                mean_color = np.mean(block, axis=(0, 1))
                mosaic_image[r:r + size, c:c + size] = mean_color
    
    return mosaic_image

def blend_images(original, mosaic, alpha=0.5):
    return cv2.addWeighted(original, alpha, mosaic, 1 - alpha, 0)

def bounding_box(image, box_color=(0, 255, 0), box_thickness=3):
    rows, cols, _ = image.shape
    box_image = np.copy(image)
    cv2.rectangle(box_image, (10, 10), (cols-10, rows-10), box_color, box_thickness)
    return box_image

def collar_jitter(image, collar_size=0.1):
    rows, cols, _ = image.shape
    collar_width = int(min(rows, cols) * collar_size)
    collar_image = np.copy(image)
    collar_image[:collar_width, :] = np.random.rand(collar_width, cols, 3)
    collar_image[-collar_width:, :] = np.random.rand(collar_width, cols, 3)
    collar_image[:, :collar_width] = np.random.rand(rows, collar_width, 3)
    collar_image[:, -collar_width:] = np.random.rand(rows, collar_width, 3)
    return collar_image

def image_warp(image, alpha=1.0):
    rows, cols, _ = image.shape
    M = np.float32([[1, alpha, 0], [0, 1, 0]])
    warped_image = cv2.warpAffine(image, M, (cols, rows), flags=cv2.INTER_LINEAR)
    return warped_image

def channel_shuffle(image):
    shuffled_image = np.copy(image)
    channels = [0, 1, 2]
    np.random.shuffle(channels)
    return shuffled_image[..., channels]

def polar_distortion(image):
    rows, cols, _ = image.shape
    center = (cols // 2, rows // 2)
    max_radius = np.sqrt((center[0] ** 2) + (center[1] ** 2))
    distorted = np.zeros_like(image)
    for r in range(rows):
        for c in range(cols):
            radius = np.sqrt((c - center[0]) ** 2 + (r - center[1]) ** 2)
            theta = np.arctan2(r - center[1], c - center[0])
            radius = radius / max_radius
            x = int(center[0] + radius * np.cos(theta) * center[0])
            y = int(center[1] + radius * np.sin(theta) * center[1])
            if 0 <= x < cols and 0 <= y < rows:
                distorted[r, c] = image[y, x]
    return distorted

def hide_and_seek(image):
    rows, cols, _ = image.shape
    hide_image = np.copy(image)
    mask = np.random.choice([0, 1], size=(rows, cols), p=[0.2, 0.8])
    hide_image[mask == 0] = 0
    return hide_image

def apply_box_grid(image, grid_size=32):
    rows, cols, _ = image.shape
    grid_image = np.copy(image)

    for r in range(0, rows, grid_size):
        for c in range(0, cols, grid_size):
            if r + grid_size <= rows and c + grid_size <= cols:
                cv2.rectangle(grid_image, (c, r), (c + grid_size, r + grid_size), (0, 1, 0), 1)
    
    return grid_image

def color_shift(image, shift_range=0.2):
    shift = np.random.uniform(-shift_range, shift_range, size=(3,))
    shifted_image = np.clip(image + shift, 0, 1)
    return shifted_image

def affine_transform(image, matrix=None):
    rows, cols, _ = image.shape
    if matrix is None:
        matrix = np.float32([[1, 0, 0], [0, 1, 0]])
    transformed_image = cv2.warpAffine(image, matrix, (cols, rows), flags=cv2.INTER_LINEAR)
    return transformed_image

def deformable_conv(image):
    kernel = np.array([[0, 0, 0], [0, 1, 0], [0, 0, 0]], dtype=np.float32)
    deformable_image = cv2.filter2D(image, -1, kernel)
    return deformable_image

def solarize(image, threshold):
    image = (image * 255).astype(np.uint8)
    solarized = cv2.bitwise_not(image) if np.mean(image) > threshold * 255 else image
    return solarized / 255.0

def invert_image(image):
    return 1.0 - image

def color_jitter(image):
    img = Image.fromarray((image * 255).astype(np.uint8))
    enhancer = ImageEnhance.Color(img)
    jittered = enhancer.enhance(np.random.uniform(0.5, 1.5))
    jittered = np.array(jittered) / 255.0
    return jittered

def sigmoid_contrast(image):
    img = (image * 255).astype(np.uint8)
    img = exposure.adjust_sigmoid(img, gain=10, cutoff=0.5)
    return img / 255.0

def gamma_contrast(image):
    gamma = np.random.uniform(0.5, 2.0)
    img = (image * 255).astype(np.uint8)
    img = exposure.adjust_gamma(img, gamma)
    return img / 255.0

def linear_contrast(image):
    img = (image * 255).astype(np.uint8)
    img = exposure.rescale_intensity(img, in_range='image', out_range='dtype')
    return img / 255.0

def perspective_transform(image, src_pts=None, dst_pts=None):
    """Apply a perspective transformation to an image."""
    h, w = image.shape[:2]
    if src_pts is None or dst_pts is None:
        src_pts = np.float32([[0, 0], [w-1, 0], [0, h-1], [w-1, h-1]])
        dst_pts = np.float32([[0, 0], [w-1, 0], [0, h-1], [w-1, h-1]])
    
    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
    transformed_image = cv2.warpPerspective(image, matrix, (w, h))
    return transformed_image

def spatial_transform(image, theta):
    rows, cols, _ = image.shape
    M = np.float32([[1, 0, 0], [0, 1, 0]])
    transformed = cv2.warpAffine(image, M, (cols, rows), flags=cv2.INTER_LINEAR)
    return transformed

def mixup(image, image2):
    if image.shape != image2.shape:
        raise ValueError("Images must have the same dimensions for mixup.")
    
    alpha = np.random.uniform(0.3, 0.7)
    
    # Make sure both images are in the range [0, 1]
    image = np.clip(image, 0, 1)
    image2 = np.clip(image2, 0, 1)
    
    mixed = alpha * image + (1 - alpha) * image2
    
    # Ensure mixed image is still in the range [0, 1]
    mixed = np.clip(mixed, 0, 1)
    
    return mixed

def random_erasing(image, scale=(0.02, 0.4), ratio=(0.3, 3.3), value=0.0):
    """
    Applies random erasing to the given image.

    Args:
        image (np.ndarray): The input image to be processed.
        scale (tuple): Range of the erasing area as a fraction of the image area.
        ratio (tuple): Aspect ratio range of the erasing area.
        value (float or int): The value to fill the erased area (for grayscale images).

    Returns:
        np.ndarray: The processed image with random erasing applied.
    """
    rows, cols, _ = image.shape
    area = rows * cols

    # Determine the target area for erasing
    target_area = np.random.uniform(scale[0], scale[1]) * area
    aspect_ratio = np.random.uniform(ratio[0], ratio[1])

    erase_w = int(np.sqrt(target_area * aspect_ratio))
    erase_h = int(np.sqrt(target_area / aspect_ratio))

    # Ensure erasing dimensions are within image size
    erase_w = min(erase_w, cols)
    erase_h = min(erase_h, rows)

    if erase_w <= 0 or erase_h <= 0:
        print("Invalid erased area dimensions. Skipping random erasing.")
        return image

    print(f"Erase dimensions: width={erase_w}, height={erase_h}")

    # Randomly select the top-left corner of the erasing area
    x1 = np.random.randint(0, cols - erase_w + 1)
    y1 = np.random.randint(0, rows - erase_h + 1)

    # Apply the erasing
    image[y1:y1 + erase_h, x1:x1 + erase_w] = value

    return image

def occlusion(image, box_size):
    h, w, _ = image.shape
    if box_size > min(h, w):
        raise ValueError("Box size must be smaller than both dimensions of the image.")
    
    x = np.random.randint(0, w - box_size)
    y = np.random.randint(0, h - box_size)
    
    # Ensure the box is within image boundaries
    x = np.clip(x, 0, w - box_size)
    y = np.clip(y, 0, h - box_size)
    
    image[y:y + box_size, x:x + box_size, :] = 0
    return image

def apply_augmentations_to_image(image, image_name, output_dir):
    augmentations = [
        ('Skew', lambda img: skew_image(img, skew_factor=np.random.uniform(-0.2, 0.2))),
        ('Rotate', lambda img: rotate_image(img, angle=np.random.uniform(-30, 30))),
        ('Translate', lambda img: translate_image(img, tx=np.random.uniform(-0.2, 0.2), ty=np.random.uniform(-0.2, 0.2))),
        ('Scale', lambda img: scale_image(img, scale=np.random.uniform(0.8, 1.2))),
        ('Shear', lambda img: shear_image(img, shear=np.random.uniform(-0.2, 0.2))),
        ('Flip', lambda img: flip_image(img, direction=np.random.choice(['horizontal', 'vertical']))),
        ('Zoom', lambda img: zoom_image(img, zoom_factor=np.random.uniform(0.8, 1.2))),
        ('GaussianNoise', lambda img: add_gaussian_noise(img, var=np.random.uniform(0.01, 0.1))),
        ('SaltAndPepper', lambda img: add_salt_and_pepper_noise(img, amount=np.random.uniform(0.01, 0.1))),
        ('Blur', lambda img: apply_gaussian_blur(img, sigma=np.random.uniform(0.5, 2.0))),
        ('Sharpen', lambda img: sharpen_image(img)),
        ('TemperatureJitter', lambda img: temperature_jitter(img, jitter_amount=np.random.uniform(-0.2, 0.2))),
        ('Occlusion', lambda img: occlusion(np.copy(img), box_size=np.random.randint(10, 50))),
        ('ElasticDistortion', lambda img: elastic_distortion(img, alpha=np.random.uniform(1, 5), sigma=np.random.uniform(0.5, 2.0))),
        ('Pseudocoloring', lambda img: pseudocoloring(img)),
        ('RandomCropping', lambda img: random_cropping(img, crop_size=(np.random.randint(50, 100), np.random.randint(50, 100)))),
        ('Mosaic', lambda img: blend_images(img, mosaic(img, size=32), alpha=0.5)),
        ('BoundingBox', lambda img: bounding_box(img)),
        ('CollarJitter', lambda img: collar_jitter(img, collar_size=np.random.uniform(0.05, 0.2))),
        ('ImageWarp', lambda img: image_warp(img, alpha=np.random.uniform(0.1, 0.5))),
        ('ChannelShuffle', lambda img: channel_shuffle(img)),
        ('Solarize', lambda img: solarize(img, threshold=np.random.uniform(0.2, 0.8))),
        ('Invert', lambda img: invert_image(img)),
        ('ColorJitter', lambda img: color_jitter(img)),
        ('SigmoidContrast', lambda img: sigmoid_contrast(img)),
        ('GammaContrast', lambda img: gamma_contrast(img)),
        ('LinearContrast', lambda img: linear_contrast(img)),
        ('PolarDistortion', lambda img: polar_distortion(img)),
        ('HideAndSeek', lambda img: hide_and_seek(img)),
        ('BoxGrid', lambda img: apply_box_grid(img, grid_size=32)),
        ('Mixup', lambda img: mixup(img, image2=np.random.rand(*img.shape))),
        ('ColorShift', lambda img: color_shift(img)),
        ('AffineTransform', lambda img: affine_transform(img)),
        ('PerspectiveTransform', lambda img: perspective_transform(img, 
             src_pts=np.float32([[0, 0], [img.shape[1]-1, 0], [0, img.shape[0]-1], [img.shape[1]-1, img.shape[0]-1]]),
             dst_pts=np.float32([[0, 0], [img.shape[1]-1, 100], [0, img.shape[0]-100], [img.shape[1]-1, img.shape[0]-100]]))),
        ('SpatialTransform', lambda img: spatial_transform(img, theta=np.random.uniform(-0.1, 0.1))),
        ('DeformableConv', lambda img: deformable_conv(img)),
        ('RandomErasing', lambda img: random_erasing(img, scale=(0.02, 0.4), ratio=(0.3, 3.3), value=0.0))
    ]
    
    for i, (aug_name, aug_fn) in enumerate(augmentations):
        try:
            augmented_image = aug_fn(np.copy(image))  # Apply augmentations on a copy of the image
            output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}_{aug_name}_{i}.jpg")
            augmented_image = (augmented_image * 255).astype(np.uint8)
            augmented_image = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(output_path, augmented_image)
            print(f"Augmented image saved to {output_path}")
        except Exception as e:
            print(f"Error applying {aug_name}: {e}")

def process_image(input_image_path, output_dir):
    try:
        image = cv2.imread(input_image_path)
        if image is None:
            raise FileNotFoundError(f"Image not found: {input_image_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0  # Convert BGR to RGB and normalize
        image_name = os.path.basename(input_image_path)
        
        # Create a random second image with the same dimensions
        image2 = np.random.rand(*image.shape)  # Create random image with same shape
        image2 = np.clip(image2, 0, 1)  # Ensure image2 is in the range [0, 1]
        
        # Apply augmentations including mixup
        apply_augmentations_to_image(image, image_name, output_dir)
        
        # Apply mixup separately
        mixed_image = mixup(image, image2)
        output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}_Mixup.jpg")
        mixed_image = (mixed_image * 255).astype(np.uint8)
        mixed_image = cv2.cvtColor(mixed_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_path, mixed_image)
        print(f"Mixup image saved to {output_path}")
    
    except Exception as e:
        print(f"Error processing image {input_image_path}: {e}")
        
def test_augmentation(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename in os.listdir(input_dir):
        input_image_path = os.path.join(input_dir, filename)
        if os.path.isfile(input_image_path):
            print(f"Processing {input_image_path}...")
            process_image(input_image_path, output_dir)

# Set the paths to your directories
input_dir = ' # Update with the actual dataset path'
output_dir = '# update with the path or folder where it has to be saved'

# Process all images in the input directory and save them to the output directory
test_augmentation(input_dir, output_dir)

In [ ]:
import argparse
import cv2
import numpy as np
from PIL import Image, ImageEnhance
from scipy.ndimage import gaussian_filter, map_coordinates
from skimage import exposure
import os
import random

# Define image augmentation functions

def skew_image(image, skew_factor):
    rows, cols, _ = image.shape
    M = np.float32([[1, skew_factor, 0], [0, 1, 0]])
    skewed = cv2.warpAffine(image, M, (cols, rows))
    return skewed

def rotate_image(image, angle):
    rows, cols, _ = image.shape
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    rotated = cv2.warpAffine(image, M, (cols, rows))
    return rotated

def translate_image(image, tx, ty):
    rows, cols, _ = image.shape
    M = np.float32([[1, 0, tx * cols], [0, 1, ty * rows]])
    translated = cv2.warpAffine(image, M, (cols, rows))
    return translated

def scale_image(image, scale):
    rows, cols, _ = image.shape
    M = np.float32([[scale, 0, 0], [0, scale, 0]])
    new_cols, new_rows = int(cols * scale), int(rows * scale)
    scaled = cv2.warpAffine(image, M, (new_cols, new_rows), borderMode=cv2.BORDER_REFLECT)
    return scaled

def shear_image(image, shear):
    rows, cols, _ = image.shape
    M = np.float32([[1, shear, 0], [0, 1, 0]])
    sheared = cv2.warpAffine(image, M, (cols, rows))
    return sheared

def flip_image(image, direction):
    if direction == 'horizontal':
        flipped = cv2.flip(image, 1)
    elif direction == 'vertical':
        flipped = cv2.flip(image, 0)
    else:
        raise ValueError("Direction must be 'horizontal' or 'vertical'")
    return flipped

def zoom_image(image, zoom_factor):
    rows, cols, _ = image.shape
    center = (cols // 2, rows // 2)
    M = cv2.getRotationMatrix2D(center, 0, zoom_factor)
    zoomed = cv2.warpAffine(image, M, (cols, rows))
    return zoomed

def add_gaussian_noise(image, var):
    """
    Add Gaussian noise to an image.

    :param image: Input image as a NumPy array with values in [0, 1].
    :param var: Variance of the Gaussian noise.
    :return: Noisy image.
    """
    mean = 0
    sigma = var**0.5
    gaussian = np.random.normal(mean, sigma, image.shape)
    noisy = np.clip(image + gaussian, 0, 1)
    return noisy

def add_salt_and_pepper_noise(image, amount=0.05):
    """
    Add salt-and-pepper noise to an image.

    :param image: Input image as a NumPy array with values in [0, 1].
    :param amount: Proportion of pixels to be replaced with salt and pepper noise.
    :return: Noisy image.
    """
    if len(image.shape) != 3:
        raise ValueError("Image must be 3-dimensional.")
    
    rows, cols, _ = image.shape
    s_vs_p = 0.5
    out = np.copy(image)
    
    # Salt noise
    num_salt = np.ceil(amount * image.size * s_vs_p)
    salt_coords = [np.random.randint(0, i, int(num_salt)) for i in image.shape[:2]]
    out[salt_coords[0], salt_coords[1]] = 1
    
    # Pepper noise
    num_pepper = np.ceil(amount * image.size * (1. - s_vs_p))
    pepper_coords = [np.random.randint(0, i, int(num_pepper)) for i in image.shape[:2]]
    out[pepper_coords[0], pepper_coords[1]] = 0
    
    return out

def apply_gaussian_blur(image, sigma):
    blurred = gaussian_filter(image, sigma=sigma)
    return blurred

def sharpen_image(image):
    if image.dtype == np.float32 or image.dtype == np.float64:
        # Normalize image if it's not in 0-255 range
        image = (image * 255).astype(np.uint8)
    
    # Define a sharpening kernel
    kernel = np.array([[0, -0.25, 0], 
                       [-0.25, 2, -0.25], 
                       [0, -0.25, 0]], dtype=np.float32)
    
    sharpened = cv2.filter2D(image, -1, kernel)
    
    if sharpened.dtype == np.uint8:
        # Normalize back to [0, 1] if necessary
        sharpened = sharpened / 255.0
    
    return sharpened

def temperature_jitter(image, jitter_amount):
    image = Image.fromarray((image * 255).astype(np.uint8))
    enhancer = ImageEnhance.Color(image)
    jittered = enhancer.enhance(1 + jitter_amount)
    jittered = np.array(jittered) / 255.0
    return jittered

def grid_mask(image, grid_size=30):
    rows, cols, _ = image.shape
    mask = np.ones((rows, cols), dtype=np.uint8)
    
    for r in range(0, rows, grid_size):
        for c in range(0, cols, grid_size):
            mask[r:min(r + grid_size, rows), c:min(c + grid_size, cols)] = 0

    grid_masked_image = np.copy(image)
    grid_masked_image[mask == 0] = 0
    return grid_masked_image

def elastic_distortion(image, alpha=1.0, sigma=0.1):
    """
    Apply elastic distortion to an image.

    :param image: Input image as a NumPy array.
    :param alpha: Scaling factor for the displacement field.
    :param sigma: Standard deviation for the Gaussian filter.
    :return: Distorted image.
    """
    shape = image.shape
    height, width = shape[:2]
    
    # Create displacement fields
    dx = gaussian_filter((np.random.rand(height, width) * 2 - 1), sigma, mode='constant', cval=0.0) * alpha
    dy = gaussian_filter((np.random.rand(height, width) * 2 - 1), sigma, mode='constant', cval=0.0) * alpha

    # Generate meshgrid
    x, y = np.meshgrid(np.arange(width), np.arange(height))
    x = x + dx
    y = y + dy
    
    # Ensure coordinates are within bounds
    x = np.clip(x, 0, width - 1)
    y = np.clip(y, 0, height - 1)
    
    # Apply elastic distortion to each channel
    distorted_image = np.zeros_like(image)
    for i in range(image.shape[2]):  # Assuming image has 3 channels
        distorted_image[..., i] = map_coordinates(image[..., i], [y.flatten(), x.flatten()], order=1, mode='reflect').reshape(image.shape[:2])

    return distorted_image

def pseudocoloring(image):
    img = (image * 255).astype(np.uint8)
    pseudocolored = cv2.applyColorMap(img, cv2.COLORMAP_JET)
    return pseudocolored / 255.0

def random_cropping(image, crop_size):
    h, w, _ = image.shape
    crop_h, crop_w = crop_size
    if crop_h > h or crop_w > w:
        raise ValueError("Crop size must be smaller than the dimensions of the image.")
    x = np.random.randint(0, w - crop_w + 1)
    y = np.random.randint(0, h - crop_h + 1)
    cropped = image[y:y + crop_h, x:x + crop_w]
    return cropped

# Placeholder functions
def mosaic(image, size=32):
    rows, cols, _ = image.shape
    mosaic_image = np.copy(image)
    
    for r in range(0, rows, size):
        for c in range(0, cols, size):
            if r + size <= rows and c + size <= cols:
                block = image[r:r + size, c:c + size]
                mean_color = np.mean(block, axis=(0, 1))
                mosaic_image[r:r + size, c:c + size] = mean_color
    
    return mosaic_image

def blend_images(original, mosaic, alpha=0.5):
    return cv2.addWeighted(original, alpha, mosaic, 1 - alpha, 0)

def bounding_box(image, box_color=(0, 255, 0), box_thickness=3):
    rows, cols, _ = image.shape
    box_image = np.copy(image)
    cv2.rectangle(box_image, (10, 10), (cols-10, rows-10), box_color, box_thickness)
    return box_image

def collar_jitter(image, collar_size=0.1):
    rows, cols, _ = image.shape
    collar_width = int(min(rows, cols) * collar_size)
    collar_image = np.copy(image)
    collar_image[:collar_width, :] = np.random.rand(collar_width, cols, 3)
    collar_image[-collar_width:, :] = np.random.rand(collar_width, cols, 3)
    collar_image[:, :collar_width] = np.random.rand(rows, collar_width, 3)
    collar_image[:, -collar_width:] = np.random.rand(rows, collar_width, 3)
    return collar_image

def image_warp(image, alpha=1.0):
    rows, cols, _ = image.shape
    M = np.float32([[1, alpha, 0], [0, 1, 0]])
    warped_image = cv2.warpAffine(image, M, (cols, rows), flags=cv2.INTER_LINEAR)
    return warped_image

def channel_shuffle(image):
    shuffled_image = np.copy(image)
    channels = [0, 1, 2]
    np.random.shuffle(channels)
    return shuffled_image[..., channels]

def polar_distortion(image):
    rows, cols, _ = image.shape
    center = (cols // 2, rows // 2)
    max_radius = np.sqrt((center[0] ** 2) + (center[1] ** 2))
    distorted = np.zeros_like(image)
    for r in range(rows):
        for c in range(cols):
            radius = np.sqrt((c - center[0]) ** 2 + (r - center[1]) ** 2)
            theta = np.arctan2(r - center[1], c - center[0])
            radius = radius / max_radius
            x = int(center[0] + radius * np.cos(theta) * center[0])
            y = int(center[1] + radius * np.sin(theta) * center[1])
            if 0 <= x < cols and 0 <= y < rows:
                distorted[r, c] = image[y, x]
    return distorted

def hide_and_seek(image):
    rows, cols, _ = image.shape
    hide_image = np.copy(image)
    mask = np.random.choice([0, 1], size=(rows, cols), p=[0.2, 0.8])
    hide_image[mask == 0] = 0
    return hide_image

def apply_box_grid(image, grid_size=32):
    rows, cols, _ = image.shape
    grid_image = np.copy(image)

    for r in range(0, rows, grid_size):
        for c in range(0, cols, grid_size):
            if r + grid_size <= rows and c + grid_size <= cols:
                cv2.rectangle(grid_image, (c, r), (c + grid_size, r + grid_size), (0, 1, 0), 1)
    
    return grid_image

def color_shift(image, shift_range=0.2):
    shift = np.random.uniform(-shift_range, shift_range, size=(3,))
    shifted_image = np.clip(image + shift, 0, 1)
    return shifted_image

def affine_transform(image, matrix=None):
    rows, cols, _ = image.shape
    if matrix is None:
        matrix = np.float32([[1, 0, 0], [0, 1, 0]])
    transformed_image = cv2.warpAffine(image, matrix, (cols, rows), flags=cv2.INTER_LINEAR)
    return transformed_image

def deformable_conv(image):
    kernel = np.array([[0, 0, 0], [0, 1, 0], [0, 0, 0]], dtype=np.float32)
    deformable_image = cv2.filter2D(image, -1, kernel)
    return deformable_image

def solarize(image, threshold):
    image = (image * 255).astype(np.uint8)
    solarized = cv2.bitwise_not(image) if np.mean(image) > threshold * 255 else image
    return solarized / 255.0

def invert_image(image):
    return 1.0 - image

def color_jitter(image):
    img = Image.fromarray((image * 255).astype(np.uint8))
    enhancer = ImageEnhance.Color(img)
    jittered = enhancer.enhance(np.random.uniform(0.5, 1.5))
    jittered = np.array(jittered) / 255.0
    return jittered

def sigmoid_contrast(image):
    img = (image * 255).astype(np.uint8)
    img = exposure.adjust_sigmoid(img, gain=10, cutoff=0.5)
    return img / 255.0

def gamma_contrast(image):
    gamma = np.random.uniform(0.5, 2.0)
    img = (image * 255).astype(np.uint8)
    img = exposure.adjust_gamma(img, gamma)
    return img / 255.0

def linear_contrast(image):
    img = (image * 255).astype(np.uint8)
    img = exposure.rescale_intensity(img, in_range='image', out_range='dtype')
    return img / 255.0

def perspective_transform(image, src_pts=None, dst_pts=None):
    """Apply a perspective transformation to an image."""
    h, w = image.shape[:2]
    if src_pts is None or dst_pts is None:
        src_pts = np.float32([[0, 0], [w-1, 0], [0, h-1], [w-1, h-1]])
        dst_pts = np.float32([[0, 0], [w-1, 0], [0, h-1], [w-1, h-1]])
    
    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
    transformed_image = cv2.warpPerspective(image, matrix, (w, h))
    return transformed_image

def spatial_transform(image, theta):
    rows, cols, _ = image.shape
    M = np.float32([[1, 0, 0], [0, 1, 0]])
    transformed = cv2.warpAffine(image, M, (cols, rows), flags=cv2.INTER_LINEAR)
    return transformed

def mixup(image, image2):
    if image.shape != image2.shape:
        raise ValueError("Images must have the same dimensions for mixup.")
    
    alpha = np.random.uniform(0.3, 0.7)
    
    # Make sure both images are in the range [0, 1]
    image = np.clip(image, 0, 1)
    image2 = np.clip(image2, 0, 1)
    
    mixed = alpha * image + (1 - alpha) * image2
    
    # Ensure mixed image is still in the range [0, 1]
    mixed = np.clip(mixed, 0, 1)
    
    return mixed

def random_erasing(image, scale=(0.02, 0.4), ratio=(0.3, 3.3), value=0.0):
    """
    Applies random erasing to the given image.

    Args:
        image (np.ndarray): The input image to be processed.
        scale (tuple): Range of the erasing area as a fraction of the image area.
        ratio (tuple): Aspect ratio range of the erasing area.
        value (float or int): The value to fill the erased area (for grayscale images).

    Returns:
        np.ndarray: The processed image with random erasing applied.
    """
    rows, cols, _ = image.shape
    area = rows * cols

    # Determine the target area for erasing
    target_area = np.random.uniform(scale[0], scale[1]) * area
    aspect_ratio = np.random.uniform(ratio[0], ratio[1])

    erase_w = int(np.sqrt(target_area * aspect_ratio))
    erase_h = int(np.sqrt(target_area / aspect_ratio))

    # Ensure erasing dimensions are within image size
    erase_w = min(erase_w, cols)
    erase_h = min(erase_h, rows)

    if erase_w <= 0 or erase_h <= 0:
        print("Invalid erased area dimensions. Skipping random erasing.")
        return image

    print(f"Erase dimensions: width={erase_w}, height={erase_h}")

    # Randomly select the top-left corner of the erasing area
    x1 = np.random.randint(0, cols - erase_w + 1)
    y1 = np.random.randint(0, rows - erase_h + 1)

    # Apply the erasing
    image[y1:y1 + erase_h, x1:x1 + erase_w] = value

    return image

def occlusion(image, box_size):
    h, w, _ = image.shape
    if box_size > min(h, w):
        raise ValueError("Box size must be smaller than both dimensions of the image.")
    
    x = np.random.randint(0, w - box_size)
    y = np.random.randint(0, h - box_size)
    
    # Ensure the box is within image boundaries
    x = np.clip(x, 0, w - box_size)
    y = np.clip(y, 0, h - box_size)
    
    image[y:y + box_size, x:x + box_size, :] = 0
    return image

def apply_augmentations_to_image(image, image_name, output_dir):
    augmentations = [
        ('Skew', lambda img: skew_image(img, skew_factor=np.random.uniform(-0.2, 0.2))),
        ('Rotate', lambda img: rotate_image(img, angle=np.random.uniform(-30, 30))),
        ('Translate', lambda img: translate_image(img, tx=np.random.uniform(-0.2, 0.2), ty=np.random.uniform(-0.2, 0.2))),
        ('Scale', lambda img: scale_image(img, scale=np.random.uniform(0.8, 1.2))),
        ('Shear', lambda img: shear_image(img, shear=np.random.uniform(-0.2, 0.2))),
        ('Flip', lambda img: flip_image(img, direction=np.random.choice(['horizontal', 'vertical']))),
        ('Zoom', lambda img: zoom_image(img, zoom_factor=np.random.uniform(0.8, 1.2))),
        ('GaussianNoise', lambda img: add_gaussian_noise(img, var=np.random.uniform(0.01, 0.1))),
        ('SaltAndPepper', lambda img: add_salt_and_pepper_noise(img, amount=np.random.uniform(0.01, 0.1))),
        ('Blur', lambda img: apply_gaussian_blur(img, sigma=np.random.uniform(0.5, 2.0))),
        ('Sharpen', lambda img: sharpen_image(img)),
        ('TemperatureJitter', lambda img: temperature_jitter(img, jitter_amount=np.random.uniform(-0.2, 0.2))),
        ('Occlusion', lambda img: occlusion(np.copy(img), box_size=np.random.randint(10, 50))),
        ('ElasticDistortion', lambda img: elastic_distortion(img, alpha=np.random.uniform(1, 5), sigma=np.random.uniform(0.5, 2.0))),
        ('Pseudocoloring', lambda img: pseudocoloring(img)),
        ('RandomCropping', lambda img: random_cropping(img, crop_size=(np.random.randint(50, 100), np.random.randint(50, 100)))),
        ('Mosaic', lambda img: blend_images(img, mosaic(img, size=32), alpha=0.5)),
        ('BoundingBox', lambda img: bounding_box(img)),
        ('CollarJitter', lambda img: collar_jitter(img, collar_size=np.random.uniform(0.05, 0.2))),
        ('ImageWarp', lambda img: image_warp(img, alpha=np.random.uniform(0.1, 0.5))),
        ('ChannelShuffle', lambda img: channel_shuffle(img)),
        ('Solarize', lambda img: solarize(img, threshold=np.random.uniform(0.2, 0.8))),
        ('Invert', lambda img: invert_image(img)),
        ('ColorJitter', lambda img: color_jitter(img)),
        ('SigmoidContrast', lambda img: sigmoid_contrast(img)),
        ('GammaContrast', lambda img: gamma_contrast(img)),
        ('LinearContrast', lambda img: linear_contrast(img)),
        ('PolarDistortion', lambda img: polar_distortion(img)),
        ('HideAndSeek', lambda img: hide_and_seek(img)),
        ('BoxGrid', lambda img: apply_box_grid(img, grid_size=32)),
        ('Mixup', lambda img: mixup(img, image2=np.random.rand(*img.shape))),
        ('ColorShift', lambda img: color_shift(img)),
        ('AffineTransform', lambda img: affine_transform(img)),
        ('PerspectiveTransform', lambda img: perspective_transform(img, 
             src_pts=np.float32([[0, 0], [img.shape[1]-1, 0], [0, img.shape[0]-1], [img.shape[1]-1, img.shape[0]-1]]),
             dst_pts=np.float32([[0, 0], [img.shape[1]-1, 100], [0, img.shape[0]-100], [img.shape[1]-1, img.shape[0]-100]]))),
        ('SpatialTransform', lambda img: spatial_transform(img, theta=np.random.uniform(-0.1, 0.1))),
        ('DeformableConv', lambda img: deformable_conv(img)),
        ('RandomErasing', lambda img: random_erasing(img, scale=(0.02, 0.4), ratio=(0.3, 3.3), value=0.0))
    ]
    
    for i, (aug_name, aug_fn) in enumerate(augmentations):
        try:
            augmented_image = aug_fn(np.copy(image))  # Apply augmentations on a copy of the image
            output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}_{aug_name}_{i}.jpg")
            augmented_image = (augmented_image * 255).astype(np.uint8)
            augmented_image = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(output_path, augmented_image)
            print(f"Augmented image saved to {output_path}")
        except Exception as e:
            print(f"Error applying {aug_name}: {e}")

def process_image(input_image_path, output_dir):
    try:
        image = cv2.imread(input_image_path)
        if image is None:
            raise FileNotFoundError(f"Image not found: {input_image_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0  # Convert BGR to RGB and normalize
        image_name = os.path.basename(input_image_path)
        
        # Create a random second image with the same dimensions
        image2 = np.random.rand(*image.shape)  # Create random image with same shape
        image2 = np.clip(image2, 0, 1)  # Ensure image2 is in the range [0, 1]
        
        # Apply augmentations including mixup
        apply_augmentations_to_image(image, image_name, output_dir)
        
        # Apply mixup separately
        mixed_image = mixup(image, image2)
        output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}_Mixup.jpg")
        mixed_image = (mixed_image * 255).astype(np.uint8)
        mixed_image = cv2.cvtColor(mixed_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_path, mixed_image)
        print(f"Mixup image saved to {output_path}")
    
    except Exception as e:
        print(f"Error processing image {input_image_path}: {e}")
        
def test_augmentation(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename in os.listdir(input_dir):
        input_image_path = os.path.join(input_dir, filename)
        if os.path.isfile(input_image_path):
            print(f"Processing {input_image_path}...")
            process_image(input_image_path, output_dir)

# Set the paths to your directories
input_dir = ' # Update with the actual dataset path'
output_dir = '# update with the path or folder where it has to be saved'

# Process all images in the input directory and save them to the output directory
test_augmentation(input_dir, output_dir)

In [ ]:
pip install pillow-heif

In [ ]:
import argparse
import cv2
import numpy as np
from PIL import Image, ImageEnhance
from scipy.ndimage import gaussian_filter, map_coordinates
from skimage import exposure
import os
import random
from PIL import Image
import pillow_heif

# Define image augmentation functions

def skew_image(image, skew_factor):
    rows, cols, _ = image.shape
    M = np.float32([[1, skew_factor, 0], [0, 1, 0]])
    skewed = cv2.warpAffine(image, M, (cols, rows))
    return skewed

def rotate_image(image, angle):
    rows, cols, _ = image.shape
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    rotated = cv2.warpAffine(image, M, (cols, rows))
    return rotated

def translate_image(image, tx, ty):
    rows, cols, _ = image.shape
    M = np.float32([[1, 0, tx * cols], [0, 1, ty * rows]])
    translated = cv2.warpAffine(image, M, (cols, rows))
    return translated

def scale_image(image, scale):
    rows, cols, _ = image.shape
    M = np.float32([[scale, 0, 0], [0, scale, 0]])
    new_cols, new_rows = int(cols * scale), int(rows * scale)
    scaled = cv2.warpAffine(image, M, (new_cols, new_rows), borderMode=cv2.BORDER_REFLECT)
    return scaled

def shear_image(image, shear):
    rows, cols, _ = image.shape
    M = np.float32([[1, shear, 0], [0, 1, 0]])
    sheared = cv2.warpAffine(image, M, (cols, rows))
    return sheared

def flip_image(image, direction):
    if direction == 'horizontal':
        flipped = cv2.flip(image, 1)
    elif direction == 'vertical':
        flipped = cv2.flip(image, 0)
    else:
        raise ValueError("Direction must be 'horizontal' or 'vertical'")
    return flipped

def zoom_image(image, zoom_factor):
    rows, cols, _ = image.shape
    center = (cols // 2, rows // 2)
    M = cv2.getRotationMatrix2D(center, 0, zoom_factor)
    zoomed = cv2.warpAffine(image, M, (cols, rows))
    return zoomed

def add_gaussian_noise(image, var):
    """
    Add Gaussian noise to an image.

    :param image: Input image as a NumPy array with values in [0, 1].
    :param var: Variance of the Gaussian noise.
    :return: Noisy image.
    """
    mean = 0
    sigma = var**0.5
    gaussian = np.random.normal(mean, sigma, image.shape)
    noisy = np.clip(image + gaussian, 0, 1)
    return noisy

def add_salt_and_pepper_noise(image, amount=0.05):
    """
    Add salt-and-pepper noise to an image.

    :param image: Input image as a NumPy array with values in [0, 1].
    :param amount: Proportion of pixels to be replaced with salt and pepper noise.
    :return: Noisy image.
    """
    if len(image.shape) != 3:
        raise ValueError("Image must be 3-dimensional.")
    
    rows, cols, _ = image.shape
    s_vs_p = 0.5
    out = np.copy(image)
    
    # Salt noise
    num_salt = np.ceil(amount * image.size * s_vs_p)
    salt_coords = [np.random.randint(0, i, int(num_salt)) for i in image.shape[:2]]
    out[salt_coords[0], salt_coords[1]] = 1
    
    # Pepper noise
    num_pepper = np.ceil(amount * image.size * (1. - s_vs_p))
    pepper_coords = [np.random.randint(0, i, int(num_pepper)) for i in image.shape[:2]]
    out[pepper_coords[0], pepper_coords[1]] = 0
    
    return out

def apply_gaussian_blur(image, sigma):
    blurred = gaussian_filter(image, sigma=sigma)
    return blurred

def sharpen_image(image):
    if image.dtype == np.float32 or image.dtype == np.float64:
        # Normalize image if it's not in 0-255 range
        image = (image * 255).astype(np.uint8)
    
    # Define a sharpening kernel
    kernel = np.array([[0, -0.25, 0], 
                       [-0.25, 2, -0.25], 
                       [0, -0.25, 0]], dtype=np.float32)
    
    sharpened = cv2.filter2D(image, -1, kernel)
    
    if sharpened.dtype == np.uint8:
        # Normalize back to [0, 1] if necessary
        sharpened = sharpened / 255.0
    
    return sharpened

def temperature_jitter(image, jitter_amount):
    image = Image.fromarray((image * 255).astype(np.uint8))
    enhancer = ImageEnhance.Color(image)
    jittered = enhancer.enhance(1 + jitter_amount)
    jittered = np.array(jittered) / 255.0
    return jittered

def grid_mask(image, grid_size=30):
    rows, cols, _ = image.shape
    mask = np.ones((rows, cols), dtype=np.uint8)
    
    for r in range(0, rows, grid_size):
        for c in range(0, cols, grid_size):
            mask[r:min(r + grid_size, rows), c:min(c + grid_size, cols)] = 0

    grid_masked_image = np.copy(image)
    grid_masked_image[mask == 0] = 0
    return grid_masked_image

def elastic_distortion(image, alpha=1.0, sigma=0.1):
    """
    Apply elastic distortion to an image.

    :param image: Input image as a NumPy array.
    :param alpha: Scaling factor for the displacement field.
    :param sigma: Standard deviation for the Gaussian filter.
    :return: Distorted image.
    """
    shape = image.shape
    height, width = shape[:2]
    
    # Create displacement fields
    dx = gaussian_filter((np.random.rand(height, width) * 2 - 1), sigma, mode='constant', cval=0.0) * alpha
    dy = gaussian_filter((np.random.rand(height, width) * 2 - 1), sigma, mode='constant', cval=0.0) * alpha

    # Generate meshgrid
    x, y = np.meshgrid(np.arange(width), np.arange(height))
    x = x + dx
    y = y + dy
    
    # Ensure coordinates are within bounds
    x = np.clip(x, 0, width - 1)
    y = np.clip(y, 0, height - 1)
    
    # Apply elastic distortion to each channel
    distorted_image = np.zeros_like(image)
    for i in range(image.shape[2]):  # Assuming image has 3 channels
        distorted_image[..., i] = map_coordinates(image[..., i], [y.flatten(), x.flatten()], order=1, mode='reflect').reshape(image.shape[:2])

    return distorted_image

def pseudocoloring(image):
    img = (image * 255).astype(np.uint8)
    pseudocolored = cv2.applyColorMap(img, cv2.COLORMAP_JET)
    return pseudocolored / 255.0

def random_cropping(image, crop_size):
    h, w, _ = image.shape
    crop_h, crop_w = crop_size
    if crop_h > h or crop_w > w:
        raise ValueError("Crop size must be smaller than the dimensions of the image.")
    x = np.random.randint(0, w - crop_w + 1)
    y = np.random.randint(0, h - crop_h + 1)
    cropped = image[y:y + crop_h, x:x + crop_w]
    return cropped

# Placeholder functions
def mosaic(image, size=32):
    rows, cols, _ = image.shape
    mosaic_image = np.copy(image)
    
    for r in range(0, rows, size):
        for c in range(0, cols, size):
            if r + size <= rows and c + size <= cols:
                block = image[r:r + size, c:c + size]
                mean_color = np.mean(block, axis=(0, 1))
                mosaic_image[r:r + size, c:c + size] = mean_color
    
    return mosaic_image

def blend_images(original, mosaic, alpha=0.5):
    return cv2.addWeighted(original, alpha, mosaic, 1 - alpha, 0)

def bounding_box(image, box_color=(0, 255, 0), box_thickness=3):
    rows, cols, _ = image.shape
    box_image = np.copy(image)
    cv2.rectangle(box_image, (10, 10), (cols-10, rows-10), box_color, box_thickness)
    return box_image

def collar_jitter(image, collar_size=0.1):
    rows, cols, _ = image.shape
    collar_width = int(min(rows, cols) * collar_size)
    collar_image = np.copy(image)
    collar_image[:collar_width, :] = np.random.rand(collar_width, cols, 3)
    collar_image[-collar_width:, :] = np.random.rand(collar_width, cols, 3)
    collar_image[:, :collar_width] = np.random.rand(rows, collar_width, 3)
    collar_image[:, -collar_width:] = np.random.rand(rows, collar_width, 3)
    return collar_image

def image_warp(image, alpha=1.0):
    rows, cols, _ = image.shape
    M = np.float32([[1, alpha, 0], [0, 1, 0]])
    warped_image = cv2.warpAffine(image, M, (cols, rows), flags=cv2.INTER_LINEAR)
    return warped_image

def channel_shuffle(image):
    shuffled_image = np.copy(image)
    channels = [0, 1, 2]
    np.random.shuffle(channels)
    return shuffled_image[..., channels]

def polar_distortion(image):
    rows, cols, _ = image.shape
    center = (cols // 2, rows // 2)
    max_radius = np.sqrt((center[0] ** 2) + (center[1] ** 2))
    distorted = np.zeros_like(image)
    for r in range(rows):
        for c in range(cols):
            radius = np.sqrt((c - center[0]) ** 2 + (r - center[1]) ** 2)
            theta = np.arctan2(r - center[1], c - center[0])
            radius = radius / max_radius
            x = int(center[0] + radius * np.cos(theta) * center[0])
            y = int(center[1] + radius * np.sin(theta) * center[1])
            if 0 <= x < cols and 0 <= y < rows:
                distorted[r, c] = image[y, x]
    return distorted

def hide_and_seek(image):
    rows, cols, _ = image.shape
    hide_image = np.copy(image)
    mask = np.random.choice([0, 1], size=(rows, cols), p=[0.2, 0.8])
    hide_image[mask == 0] = 0
    return hide_image

def apply_box_grid(image, grid_size=32):
    rows, cols, _ = image.shape
    grid_image = np.copy(image)

    for r in range(0, rows, grid_size):
        for c in range(0, cols, grid_size):
            if r + grid_size <= rows and c + grid_size <= cols:
                cv2.rectangle(grid_image, (c, r), (c + grid_size, r + grid_size), (0, 1, 0), 1)
    
    return grid_image

def color_shift(image, shift_range=0.2):
    shift = np.random.uniform(-shift_range, shift_range, size=(3,))
    shifted_image = np.clip(image + shift, 0, 1)
    return shifted_image

def affine_transform(image, matrix=None):
    rows, cols, _ = image.shape
    if matrix is None:
        matrix = np.float32([[1, 0, 0], [0, 1, 0]])
    transformed_image = cv2.warpAffine(image, matrix, (cols, rows), flags=cv2.INTER_LINEAR)
    return transformed_image

def deformable_conv(image):
    kernel = np.array([[0, 0, 0], [0, 1, 0], [0, 0, 0]], dtype=np.float32)
    deformable_image = cv2.filter2D(image, -1, kernel)
    return deformable_image

def solarize(image, threshold):
    image = (image * 255).astype(np.uint8)
    solarized = cv2.bitwise_not(image) if np.mean(image) > threshold * 255 else image
    return solarized / 255.0

def invert_image(image):
    return 1.0 - image

def color_jitter(image):
    img = Image.fromarray((image * 255).astype(np.uint8))
    enhancer = ImageEnhance.Color(img)
    jittered = enhancer.enhance(np.random.uniform(0.5, 1.5))
    jittered = np.array(jittered) / 255.0
    return jittered

def sigmoid_contrast(image):
    img = (image * 255).astype(np.uint8)
    img = exposure.adjust_sigmoid(img, gain=10, cutoff=0.5)
    return img / 255.0

def gamma_contrast(image):
    gamma = np.random.uniform(0.5, 2.0)
    img = (image * 255).astype(np.uint8)
    img = exposure.adjust_gamma(img, gamma)
    return img / 255.0

def linear_contrast(image):
    img = (image * 255).astype(np.uint8)
    img = exposure.rescale_intensity(img, in_range='image', out_range='dtype')
    return img / 255.0

def perspective_transform(image, src_pts=None, dst_pts=None):
    """Apply a perspective transformation to an image."""
    h, w = image.shape[:2]
    if src_pts is None or dst_pts is None:
        src_pts = np.float32([[0, 0], [w-1, 0], [0, h-1], [w-1, h-1]])
        dst_pts = np.float32([[0, 0], [w-1, 0], [0, h-1], [w-1, h-1]])
    
    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
    transformed_image = cv2.warpPerspective(image, matrix, (w, h))
    return transformed_image

def spatial_transform(image, theta):
    rows, cols, _ = image.shape
    M = np.float32([[1, 0, 0], [0, 1, 0]])
    transformed = cv2.warpAffine(image, M, (cols, rows), flags=cv2.INTER_LINEAR)
    return transformed

def mixup(image, image2):
    if image.shape != image2.shape:
        raise ValueError("Images must have the same dimensions for mixup.")
    
    alpha = np.random.uniform(0.3, 0.7)
    
    # Make sure both images are in the range [0, 1]
    image = np.clip(image, 0, 1)
    image2 = np.clip(image2, 0, 1)
    
    mixed = alpha * image + (1 - alpha) * image2
    
    # Ensure mixed image is still in the range [0, 1]
    mixed = np.clip(mixed, 0, 1)
    
    return mixed

def random_erasing(image, scale=(0.02, 0.4), ratio=(0.3, 3.3), value=0.0):
    """
    Applies random erasing to the given image.

    Args:
        image (np.ndarray): The input image to be processed.
        scale (tuple): Range of the erasing area as a fraction of the image area.
        ratio (tuple): Aspect ratio range of the erasing area.
        value (float or int): The value to fill the erased area (for grayscale images).

    Returns:
        np.ndarray: The processed image with random erasing applied.
    """
    rows, cols, _ = image.shape
    area = rows * cols

    # Determine the target area for erasing
    target_area = np.random.uniform(scale[0], scale[1]) * area
    aspect_ratio = np.random.uniform(ratio[0], ratio[1])

    erase_w = int(np.sqrt(target_area * aspect_ratio))
    erase_h = int(np.sqrt(target_area / aspect_ratio))

    # Ensure erasing dimensions are within image size
    erase_w = min(erase_w, cols)
    erase_h = min(erase_h, rows)

    if erase_w <= 0 or erase_h <= 0:
        print("Invalid erased area dimensions. Skipping random erasing.")
        return image

    print(f"Erase dimensions: width={erase_w}, height={erase_h}")

    # Randomly select the top-left corner of the erasing area
    x1 = np.random.randint(0, cols - erase_w + 1)
    y1 = np.random.randint(0, rows - erase_h + 1)

    # Apply the erasing
    image[y1:y1 + erase_h, x1:x1 + erase_w] = value

    return image

def occlusion(image, box_size):
    h, w, _ = image.shape
    if box_size > min(h, w):
        raise ValueError("Box size must be smaller than both dimensions of the image.")
    
    x = np.random.randint(0, w - box_size)
    y = np.random.randint(0, h - box_size)
    
    # Ensure the box is within image boundaries
    x = np.clip(x, 0, w - box_size)
    y = np.clip(y, 0, h - box_size)
    
    image[y:y + box_size, x:x + box_size, :] = 0
    return image

def apply_augmentations_to_image(image, image_name, output_dir):
    augmentations = [
        ('Skew', lambda img: skew_image(img, skew_factor=np.random.uniform(-0.2, 0.2))),
        ('Rotate', lambda img: rotate_image(img, angle=np.random.uniform(-30, 30))),
        ('Translate', lambda img: translate_image(img, tx=np.random.uniform(-0.2, 0.2), ty=np.random.uniform(-0.2, 0.2))),
        ('Scale', lambda img: scale_image(img, scale=np.random.uniform(0.8, 1.2))),
        ('Shear', lambda img: shear_image(img, shear=np.random.uniform(-0.2, 0.2))),
        ('Flip', lambda img: flip_image(img, direction=np.random.choice(['horizontal', 'vertical']))),
        ('Zoom', lambda img: zoom_image(img, zoom_factor=np.random.uniform(0.8, 1.2))),
        ('GaussianNoise', lambda img: add_gaussian_noise(img, var=np.random.uniform(0.01, 0.1))),
        ('SaltAndPepper', lambda img: add_salt_and_pepper_noise(img, amount=np.random.uniform(0.01, 0.1))),
        ('Blur', lambda img: apply_gaussian_blur(img, sigma=np.random.uniform(0.5, 2.0))),
        ('Sharpen', lambda img: sharpen_image(img)),
        ('TemperatureJitter', lambda img: temperature_jitter(img, jitter_amount=np.random.uniform(-0.2, 0.2))),
        ('Occlusion', lambda img: occlusion(np.copy(img), box_size=np.random.randint(10, 50))),
        ('ElasticDistortion', lambda img: elastic_distortion(img, alpha=np.random.uniform(1, 5), sigma=np.random.uniform(0.5, 2.0))),
        ('Pseudocoloring', lambda img: pseudocoloring(img)),
        ('RandomCropping', lambda img: random_cropping(img, crop_size=(np.random.randint(50, 100), np.random.randint(50, 100)))),
        ('Mosaic', lambda img: blend_images(img, mosaic(img, size=32), alpha=0.5)),
        ('BoundingBox', lambda img: bounding_box(img)),
        ('CollarJitter', lambda img: collar_jitter(img, collar_size=np.random.uniform(0.05, 0.2))),
        ('ImageWarp', lambda img: image_warp(img, alpha=np.random.uniform(0.1, 0.5))),
        ('ChannelShuffle', lambda img: channel_shuffle(img)),
        ('Solarize', lambda img: solarize(img, threshold=np.random.uniform(0.2, 0.8))),
        ('Invert', lambda img: invert_image(img)),
        ('ColorJitter', lambda img: color_jitter(img)),
        ('SigmoidContrast', lambda img: sigmoid_contrast(img)),
        ('GammaContrast', lambda img: gamma_contrast(img)),
        ('LinearContrast', lambda img: linear_contrast(img)),
        ('PolarDistortion', lambda img: polar_distortion(img)),
        ('HideAndSeek', lambda img: hide_and_seek(img)),
        ('BoxGrid', lambda img: apply_box_grid(img, grid_size=32)),
        ('Mixup', lambda img: mixup(img, image2=np.random.rand(*img.shape))),
        ('ColorShift', lambda img: color_shift(img)),
        ('AffineTransform', lambda img: affine_transform(img)),
        ('PerspectiveTransform', lambda img: perspective_transform(img, 
             src_pts=np.float32([[0, 0], [img.shape[1]-1, 0], [0, img.shape[0]-1], [img.shape[1]-1, img.shape[0]-1]]),
             dst_pts=np.float32([[0, 0], [img.shape[1]-1, 100], [0, img.shape[0]-100], [img.shape[1]-1, img.shape[0]-100]]))),
        ('SpatialTransform', lambda img: spatial_transform(img, theta=np.random.uniform(-0.1, 0.1))),
        ('DeformableConv', lambda img: deformable_conv(img)),
        ('RandomErasing', lambda img: random_erasing(img, scale=(0.02, 0.4), ratio=(0.3, 3.3), value=0.0))
    ]
    
    for i, (aug_name, aug_fn) in enumerate(augmentations):
        try:
            augmented_image = aug_fn(np.copy(image))  # Apply augmentations on a copy of the image
            output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}_{aug_name}_{i}.jpg")
            augmented_image = (augmented_image * 255).astype(np.uint8)
            augmented_image = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(output_path, augmented_image)
            print(f"Augmented image saved to {output_path}")
        except Exception as e:
            print(f"Error applying {aug_name}: {e}")

def process_image(input_image_path, output_dir):
    try:
        # Read HEIC image using Pillow
        pillow_heif.register_heif_opener()  # Register the HEIC opener
        image = Image.open(input_image_path)
        image = np.array(image) / 255.0  # Convert PIL image to NumPy array and normalize
        if image.ndim == 2:  # If grayscale, convert to RGB
            image = np.stack([image] * 3, axis=-1)
        image_name = os.path.basename(input_image_path)
        
        # Create a random second image with the same dimensions
        image2 = np.random.rand(*image.shape)  # Create random image with same shape
        image2 = np.clip(image2, 0, 1)  # Ensure image2 is in the range [0, 1]
        
        # Apply augmentations including mixup
        apply_augmentations_to_image(image, image_name, output_dir)
        
        # Apply mixup separately
        mixed_image = mixup(image, image2)
        output_path = os.path.join(output_dir, f"{os.path.splitext(image_name)[0]}_Mixup.jpg")
        mixed_image = (mixed_image * 255).astype(np.uint8)
        mixed_image = cv2.cvtColor(mixed_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(output_path, mixed_image)
        print(f"Mixup image saved to {output_path}")
    
    except Exception as e:
        print(f"Error processing image {input_image_path}: {e}")
        
def test_augmentation(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename in os.listdir(input_dir):
        input_image_path = os.path.join(input_dir, filename)
        if os.path.isfile(input_image_path):
            print(f"Processing {input_image_path}...")
            process_image(input_image_path, output_dir)

# Set the paths to your directories
input_dir = ' # Update with the actual dataset path'
output_dir = '# update with the path or folder where it has to be saved'

# Process all images in the input directory and save them to the output directory
test_augmentation(input_dir, output_dir)